<a href="https://colab.research.google.com/github/jjack151/Coding-Challenge/blob/master/Colab_Notebooks/CNN_Data_Provenance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Libary Imports, Google Drive Mount, and Main Directories**

In [ ]:
# imports Path library and drive library in order to mount google drive to Colab
from pathlib import Path
from google.colab import drive
drive.mount('/content/gdrive')

#imports libraries
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.layers import Dense, Flatten, Conv2D
from keras import Model
from keras import backend as K
from keras.callbacks import CSVLogger
from keras.preprocessing.image import ImageDataGenerator
import os
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import shutil
import seaborn as sns
from math import sqrt, floor, ceil
import csv
from collections import defaultdict
from tqdm import tqdm

### CODE NOT IN USE ###
# Load the TensorBoard notebook extension
#%load_ext tensorboard

Mounted at /content/gdrive


In [ ]:
#Loads compressed folder of latest upload of datasets from TrojAI notebook into google colab and uncompresses the folder 
!cp /content/gdrive/MyDrive/DataPro_TrojAI/latest.tar.gz /content/
!tar xf /content/latest.tar.gz

In [ ]:
#Directories where generated datasets from TrojAI will be stored
TrojAI_mnist_dir = Path('/content/data/TrojAI_generated_datasets/mnist_datasets/')
TrojAI_cifar10_dir = Path('/content/data/TrojAI_generated_datasets/cifar10_datasets/')

#Google drive directory for Data Provenance project where training provenance will be stored
dir = Path('/content/gdrive/My Drive/DataProvenance/')
dir.mkdir(exist_ok=True, parents=True)

#Directories where datasets for experiments will be stored
mnist_dir = Path('/content/data/experiment_datasets/mnist_datasets/')
mnist_dir.mkdir(exist_ok=True, parents=True)
cifar10_dir = Path('/content/data/experiment_datasets/cifar10_datasets/')
cifar10_dir.mkdir(exist_ok=True, parents=True)

# **Defined Functions for Training CNN Model on Clean and Triggered Datasets**

# **Data Augmentation Functions**

In [ ]:
#Function that takes directory where images are stored under labeled subdirectories as parameter
#Returns a dataframe with filepaths and labels
def make_dataframe(sdir):
    # sdir is the directory when the class subdirectories are stored
    filepaths=[]
    labels=[]
    classlist=sorted(os.listdir(sdir) )     
    for klass in classlist:
        classpath=os.path.join(sdir, klass) 
        if os.path.isdir(classpath):
            flist=sorted(os.listdir(classpath)) 
            desc=f'{klass:25s}'
            for f in tqdm(flist, ncols=130,desc=desc, unit='files', colour='blue'):
                fpath=os.path.join(classpath,f)
                filepaths.append(fpath)
                labels.append(klass)
    Fseries=pd.Series(filepaths, name='filepaths')
    Lseries=pd.Series(labels, name='labels')
    df=pd.concat([Fseries, Lseries], axis=1) 
    # return a dataframe with columns filepaths, labels
    return df

#Function that generates augmented images and stores them in designated augmented directory
def make_and_store_images(df, augdir, n,  img_size,  color_mode='rgb', save_prefix='aug-',save_format='png'):
    #augdir is the full path where augmented images will be stored
    #n is the number of augmented images that will be created for each class that has less than n image samples
    # img_size  is a tupple(height,width) that specifies the size of the augmented images
    # color_mode is 'rgb by default'
    # save_prefix is the prefix augmented images are identified with by default it is 'aug-'
    #save_format is the format augmented images will be save in, by default it is 'jpg'
    # see documentation of ImageDataGenerator at https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator for details
    df=df.copy()        
    if os.path.isdir(augdir):# start with an empty directory
        shutil.rmtree(augdir)
    os.mkdir(augdir)  # if directory does not exist create it      
    for label in df['labels'].unique():    
        classpath=os.path.join(augdir,label)    
        os.mkdir(classpath) # make class directories within aug directory
    # create and store the augmented images  
    total=0
    # in ImageDateGenerator select the types of augmentation you desire  below are some examples  
    gen=ImageDataGenerator(width_shift_range=.1, height_shift_range=.1) #MNIST / CIFAR-10

    groups=df.groupby('labels') # group by class
    for label in df['labels'].unique():  # for every class
        classdir=os.path.join(augdir, label)
        group=groups.get_group(label)  # a dataframe holding only rows with the specified label 
        sample_count=len(group)   # determine how many samples there are in this class  
        if sample_count< n: # if the class has less than target number of images
            aug_img_count=0
            delta=n - sample_count  # number of augmented images to create            
            msg='{0:40s} for class {1:^30s} creating {2:^5s} augmented images'.format(' ', label, str(delta))
            print(msg, '\r', end='') # prints over on the same line
            aug_gen=gen.flow_from_dataframe( group,  x_col='filepaths', y_col=None, target_size=img_size,
                                            class_mode=None, batch_size=1, shuffle=False, 
                                            save_to_dir=classdir, save_prefix=save_prefix, color_mode=color_mode,
                                            save_format=save_format)
            while aug_img_count<delta:
                images=next(aug_gen)            
                aug_img_count += len(images)
            total +=aug_img_count        
    print('Total Augmented images created= ', total)

In [ ]:
def normal_round(n):
    if n - floor(n) < 0.5:
        return floor(n)
    return ceil(n)

In [ ]:
#Function for getting num of samples for each label.
#Dataframes passed as parameters
#Returns list of samples for each label for both train and test datasets
def get_num_samples_perclass(clean_train, clean_test):
  
  totalTrain_count = []
  totalTest_count = []

  for label in clean_train['true_label'].unique():

    label_df = clean_train[clean_train['true_label'] == label]
    totalTrain_count.append(label_df.shape[0])       
  
  for label in clean_test['true_label'].unique():

    label_df = clean_test[clean_test['true_label'] == label]
    totalTest_count.append(label_df.shape[0])       
  
  return totalTrain_count, totalTest_count

In [ ]:
#Function that generates augmented images and stores them in corresponding directories.
#Returns directories where one will need to pull these images later so that they can be added to original dataset
def generate_augmented_datasets(TrojAI_dir, ds_dir, src_dir1, src_dir2, aug_nTrain_samples, aug_nTest_samples, img_size, color_mode):

  df1 = make_dataframe(src_dir1)
  df2 = make_dataframe(src_dir2)

  aug_cleanTrain_dir = TrojAI_dir.joinpath('{}_augmented/train/'.format(ds_dir))
  aug_cleanTrain_dir.mkdir(exist_ok=True, parents=True)
  aug_cleanTest_dir = TrojAI_dir.joinpath('{}_augmented/test/'.format(ds_dir))
  aug_cleanTest_dir.mkdir(exist_ok=True, parents=True)

  #make_and_store_images(df, augdir, n,  img_size,  color_mode='rgb', save_prefix='aug-',save_format='png'
    #augdir is the full path where augmented images will be stored
    # n is the number of augmented images that will be created for each class that has less than n image samples
    # img_size  is a tupple(height,width) that specifies the size of the augmented images
    # color_mode is 'rgb by default'
    # save_prefix is the prefix augmented images are identified with by default it is 'aug-'
    #save_format is the format augmented images will be save in, by default it is 'png'
    # see documentation of ImageDataGenerator at https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator for details

  make_and_store_images(df1, aug_cleanTrain_dir, aug_nTrain_samples, img_size, color_mode)
  #Take label with highest sample count and times by highest percentage that will be adding into training set
  make_and_store_images(df2, aug_cleanTest_dir, aug_nTest_samples, img_size, color_mode)

  return aug_cleanTrain_dir, aug_cleanTest_dir


In [ ]:
#Function for moving certain number of augmented images from its directory into corresponding experiment directories based on augmented percentage
def mv_aug_images_to_experiment_datasets_all_to_all(aug_cleanTrain_dir, aug_cleanTest_dir, aug_TrainE_dir, aug_TestE_dir, aug_percentage):

  src_df1 = make_dataframe(aug_cleanTrain_dir)
  src_df2 = make_dataframe(aug_cleanTest_dir)

  dest_df1 = make_dataframe(aug_TrainE_dir)
  dest_df2 = make_dataframe(aug_TestE_dir)  
  
  groups1 = dest_df1.groupby('labels') # group by class
  groups2 = dest_df2.groupby('labels') # group by class

  aug_percent = float(aug_percentage)/100

  all_to_all_train = []
  all_to_all_test = []

  for label in dest_df1['labels'].unique():  # for every class
  
    group = groups1.get_group(label)  # a dataframe holding only rows with the specified label 
    sample_count=len(group)   # determine how many samples there are in this class

    rows_to_move = normal_round((sample_count * aug_percent))

    all_to_all_train.append(rows_to_move)

  all_to_all_train.insert(0, all_to_all_train.pop())
  index1 = 0

  for label in src_df1['labels'].unique():

    tmp_df = src_df1[src_df1['labels'] == label]
    tmp_df = tmp_df.sample(all_to_all_train[index1]) 

    for idx, row in tmp_df.iterrows():

      my_file = row['filepaths']
      to_file = aug_TrainE_dir.joinpath(str(row["labels"]))

      #Checks if image file already exist in label subdir. If not, moves image in corresponding subdir. If it exist, prints message notifying user.
      if not (to_file.is_file()):
        shutil.copy(my_file, to_file)
      else:
        print("File already exist in destination:" + str(to_file))

    index1 = index1 + 1

  for label in dest_df2['labels'].unique():  # for every class
  
    group = groups2.get_group(label)  # a dataframe holding only rows with the specified label 
    sample_count=len(group)   # determine how many samples there are in this class

    rows_to_move = normal_round((sample_count * aug_percent))

    all_to_all_test.append(rows_to_move)

  all_to_all_test.insert(0, all_to_all_test.pop())

  index2 = 0

  for label in src_df2['labels'].unique():

    tmp_df = src_df2[src_df2['labels'] == label]
    tmp_df = tmp_df.sample(all_to_all_test[index2]) 

    for idx, row in tmp_df.iterrows():
      
      my_file = row['filepaths']
      to_file = aug_TestE_dir.joinpath(str(row["labels"]))

      #Checks if image file already exist in label subdir. If not, moves image in corresponding subdir. If it exist, prints message notifying user.
      if not (to_file.is_file()):
        shutil.copy(my_file, to_file)
      else:
        print("File already exist in destination:" + str(to_file))      

    index2 = index2 + 1

In [ ]:
#Function for moving certain number of augmented images from its directory into corresponding experiment directories based on augmented percentage
def mv_aug_images_to_experiment_datasets_single_target(aug_cleanTrain_dir, aug_cleanTest_dir, aug_TrainE_dir, aug_TestE_dir, aug_percentage, Truth_Label, Target_Label):

  src_df1 = make_dataframe(aug_cleanTrain_dir)
  src_df2 = make_dataframe(aug_cleanTest_dir)

  dest_df1 = make_dataframe(aug_TrainE_dir)
  dest_df2 = make_dataframe(aug_TestE_dir)  
  
  groups1 = dest_df1.groupby('labels') # group by class
  groups2 = dest_df2.groupby('labels') # group by class

  aug_percent = float(aug_percentage)/100

  dest_df1[dest_df1['labels'] == Truth_Label]# for Target Label
  
  group = groups1.get_group(Truth_Label)  # a dataframe holding only rows with the specified label 
  sample_count=len(group)   # determine how many samples there are in this class

  rows_to_move = normal_round((sample_count * aug_percent))

  tmp_df1 = src_df1[src_df1['labels'] == Target_Label]
  tmp_df1 = tmp_df1.sample(rows_to_move) 

  for idx, row in tmp_df1.iterrows():

    my_file = row['filepaths']
    to_file = aug_TrainE_dir.joinpath(str(row["labels"]))

    #Checks if image file already exist in label subdir. If not, moves image in corresponding subdir. If it exist, prints message notifying user.
    if not (to_file.is_file()):
      shutil.copy(my_file, to_file)
    else:
      print("File already exist in destination:" + str(to_file))
  
  group = groups2.get_group(Truth_Label)  # a dataframe holding only rows with the specified label 
  sample_count=len(group)   # determine how many samples there are in this class

  rows_to_move = normal_round((sample_count * aug_percent))

  tmp_df2 = src_df2[src_df2['labels'] == Target_Label]
  tmp_df2 = tmp_df2.sample(rows_to_move) 

  for idx, row in tmp_df2.iterrows():
      
    my_file = row['filepaths']
    to_file = aug_TestE_dir.joinpath(str(row["labels"]))

    #Checks if image file already exist in label subdir. If not, moves image in corresponding subdir. If it exist, prints message notifying user.
    if not (to_file.is_file()):
      shutil.copy(my_file, to_file)
    else:
      print("File already exist in destination:" + str(to_file)) 

# **Clean and Triggered Dataset Directory Functions**

In [ ]:
#Function for setting up original clean directory for preprocessing image dataset using Tensorflow
#Dataset directories including where TrojAI generated data is stored and directory that will be used for experiemnt, labels, and corresponding CSVs passed as parameters
def setup_clean_subdir(TrojAI_dir, dataset_dir, cleanTrain_dir, cleanTest_dir, labels, clean_train, clean_test):

  list_dir= [cleanTrain_dir, cleanTest_dir]

  #Iterates through labels for given dataset and makes subdirectories based on the label under each main directory (cleanTrain, cleanTest, triggeredTrain, triggeredTest)
  for label in labels:

    for x in range(len(list_dir)):
      #Create subdir for labels under clean and triggered directories
      label_subdir = list_dir[x].joinpath(str(labels[label]))
      label_subdir.mkdir(exist_ok=True, parents=True)

  #Iterates through clean training data CSV file and moves images into corresponding label subdir under cleanTrain dir
  for index, row in clean_train.iterrows():

    my_file = TrojAI_dir.joinpath(row['file'])
    to_file = cleanTrain_dir.joinpath(str(row["train_label"])).joinpath(my_file.name)

    #Checks if image file already exist in label subdir. If not, moves image in corresponding subdir. If it exist, prints message notifying user.
    if not (to_file.is_file()):
      shutil.copy(my_file, to_file)
    else:
      print("File already exist in destination:" + str(to_file))

  #Iterates through clean test data CSV file and moves images into corresponding label subdir under cleanTest dir
  for index, row in clean_test.iterrows():

    my_file = TrojAI_dir.joinpath(row['file'])
    to_file = cleanTest_dir.joinpath(str(row["train_label"])).joinpath(my_file.name)

    #Checks if image file already exist in label subdir. If not, moves image in corresponding subdir. If it exist, prints message notifying user.
    if not (to_file.is_file()):
      shutil.copy(my_file, to_file)
    else:
      print("File already exist in destination:" + str(to_file))

In [ ]:
#Function for creating and retrieving clean original directory under main dataset directory
#Returns clean train and test directories
def get_main_clean_dir(TrojAI_dir, dataset_dir, ds_dir, labels, clean_train, clean_test,):

  #Initializes clean directories
  cleanTrain_dir = dataset_dir.joinpath('{}_clean/train/'.format(ds_dir))
  cleanTrain_dir.mkdir(exist_ok=True, parents=True)
  cleanTest_dir = dataset_dir.joinpath('{}_clean/test/'.format(ds_dir))
  cleanTest_dir.mkdir(exist_ok=True, parents=True)

  #Calls function to setup labels as subdir under clean directories
  setup_clean_subdir(TrojAI_dir, dataset_dir, cleanTrain_dir, cleanTest_dir, labels, clean_train, clean_test)

  return cleanTrain_dir, cleanTest_dir

In [ ]:
#Function for copying original clean dataset into directory that will hold original dataset plus a certain number of augmented images based on a percentage of original
def generate_clean_dirs(ds_dir, cleanTrain_dir, cleanTest_dir, aug_percentage):

  # Source path 
  src1 = cleanTrain_dir
   
  # Destination path 
  dest1 = '/content/data/experiment_datasets/{}_datasets/{}_clean_{}/train/'.format(ds_dir, ds_dir, aug_percentage)
   
  # Copy the content of 
  # source to destination 
  destination1 = shutil.copytree(src1, dest1) 

  # Source path 
  src2 = cleanTest_dir
   
  # Destination path 
  dest2 = '/content/data/experiment_datasets/{}_datasets/{}_clean_{}/test/'.format(ds_dir, ds_dir, aug_percentage)
   
  # Copy the content of 
  # source to destination 
  destination2 = shutil.copytree(src2, dest2) 

In [ ]:
#Function that gets clean directories with augmented images
#Returns Augmented Dataset Directories that will be used for experiments
def get_clean_dirs_w_aug_images(aug_cleanTrain_dir, aug_cleanTest_dir, ds_dir, cleanTrain_dir, cleanTest_dir, aug_percentage, Truth_Label=None, Target_Label='All_to_All'):

  aug_trainExperiment_dir = Path('/content/data/experiment_datasets/{}_datasets/{}_clean_{}/train/'.format(ds_dir, ds_dir, aug_percentage))
  aug_testExperiment_dir = Path('/content/data/experiment_datasets/{}_datasets/{}_clean_{}/test/'.format(ds_dir, ds_dir, aug_percentage))

  if((os.path.isdir(aug_trainExperiment_dir)) & (os.path.isdir(aug_testExperiment_dir))):
    
    #Removes directories if they already exist. This ensures that on reruns directories contain appropriate number of augmented images on top of original dataset through resetting process
    shutil. rmtree(aug_trainExperiment_dir) 
    shutil. rmtree(aug_testExperiment_dir) 
    
  #If directories do not exist, generate them and move augmented images into corresponding subdirs
  generate_clean_dirs(ds_dir, cleanTrain_dir, cleanTest_dir, aug_percentage)

  print('\nCopying Augmented Images into Clean {} Dataset ({}% Augmented) Directory ...\n'.format(ds_dir.upper(), aug_percentage))
  
  if(Target_Label == 'All_to_All'):

    mv_aug_images_to_experiment_datasets_all_to_all(aug_cleanTrain_dir, aug_cleanTest_dir, aug_trainExperiment_dir, aug_testExperiment_dir, aug_percentage)
  
  else:

    mv_aug_images_to_experiment_datasets_single_target(aug_cleanTrain_dir, aug_cleanTest_dir, aug_trainExperiment_dir, aug_testExperiment_dir, aug_percentage, Truth_Label, Target_Label)

  return aug_trainExperiment_dir, aug_testExperiment_dir

In [ ]:
#Function for setting up/creating triggered directories for preprocessing image dataset using Tensorflow
#Original clean dataset directories and triggered percentage passed as parameters
def setup_triggered_dir(ds_dir, cleanTrain_dir, cleanTest_dir, triggered_percentage):

  # Source path 
  src1 = cleanTrain_dir
   
  # Destination path 
  dest1 = '/content/data/experiment_datasets/{}_datasets/{}_triggered_{}/train/'.format(ds_dir, ds_dir, triggered_percentage)
   
  # Copy the content of 
  # source to destination 
  destination1 = shutil.copytree(src1, dest1) 

  # Source path 
  src2 = cleanTest_dir
   
  # Destination path 
  dest2 = '/content/data/experiment_datasets/{}_datasets/{}_triggered_{}/test/'.format(ds_dir, ds_dir, triggered_percentage)
   
  # Copy the content of 
  # source to destination 
  destination2 = shutil.copytree(src2, dest2) 

In [ ]:
#Function for moving triggered images into corresponding triggered directories based on triggered percentage for preprocessing image dataset using Tensorflow
#TrojAI_dir where triggered images are stored, Triggered Directories where triggered images need to be moved to, and corresponding CSVs passed as parameters
def mv_triggered_images_into_triggered_dirs(TrojAI_dir, triggeredTrain_dir, triggeredTest_dir, triggered_train, triggered_test):

  #Iterates through triggered train data CSV file and moves both clean and triggered images into corresponding label subdir under triggered Train dir
  for index, row in triggered_train.iterrows():

   if(row["triggered"] is True):
      my_file = TrojAI_dir.joinpath(row['file'])
      to_file = triggeredTrain_dir.joinpath(str(row["train_label"]))
       
      #Checks if image file already exist in label subdir. If not, moves image in corresponding subdir. If it exist, prints message notifying user.
      if not (to_file.is_file()):
        shutil.copy(my_file, to_file)
      else: 
        print("File already exist in destination:" + str(to_file))

  #Iterates through triggered test data CSV file and moves triggered test images into corresponding label subdir under triggeredTest dir
  for index, row in triggered_test.iterrows():
      
   my_file = TrojAI_dir.joinpath(row['file'])
   to_file = triggeredTest_dir.joinpath(str(row["train_label"]))

   #Checks if image file already exist in label subdir. If not, moves image in corresponding subdir. If it exist, prints message notifying user.
   if not (to_file.is_file()):
      shutil.copy(my_file, to_file)
   else: 
      print("File already exist in destination:" + str(to_file))

In [ ]:
#Function for creating and retrieving triggered directories under main dataset directory
#Returns triggered train and test directories that will be used for experiments 
def get_triggered_dirs(TrojAI_dir, cleanTrain_dir, cleanTest_dir, ds_dir, triggered_percentage, triggered_train, triggered_test):

  triggeredTrain_Experiment_dir = Path('/content/data/experiment_datasets/{}_datasets/{}_triggered_{}/train/'.format(ds_dir, ds_dir, triggered_percentage))
  triggeredTest_Experiment_dir = Path('/content/data/experiment_datasets/{}_datasets/{}_triggered_{}/test/'.format(ds_dir, ds_dir, triggered_percentage))

  if((os.path.isdir(triggeredTrain_Experiment_dir)) & (os.path.isdir(triggeredTest_Experiment_dir))):
    
    #Removes directories if they already exist. This ensures that on reruns directories contain appropriate number of triggered images on top of original dataset through resetting process
    shutil. rmtree(triggeredTrain_Experiment_dir) 
    shutil. rmtree(triggeredTest_Experiment_dir) 
    
  setup_triggered_dir(ds_dir, cleanTrain_dir, cleanTest_dir, triggered_percentage)

  print('\nCopying Triggered Images into Triggered {} Dataset ({}% Triggered) Directory ...\n'.format(ds_dir.upper(), triggered_percentage))
  mv_triggered_images_into_triggered_dirs(TrojAI_dir, triggeredTrain_Experiment_dir, triggeredTest_Experiment_dir, triggered_train, triggered_test)

  return triggeredTrain_Experiment_dir, triggeredTest_Experiment_dir

# **Dataset from Directory Function**

In [ ]:
#Function for building dataset through image dataset preprocessing from directory using Tensorflow
#Takes directory path, and related characteristics for given dataset as parameters
#Returns Tensorflow Dataset
def get_dataset(dir_path, img_height, img_width, ds_color_mode, ds_batch_size):

  dataset = tf.keras.preprocessing.image_dataset_from_directory(
      dir_path,
      labels="inferred",
      label_mode="int",
      color_mode=ds_color_mode,
      image_size=(img_height, img_width),
      shuffle=True,
      validation_split=None,
      subset=None,
      interpolation="bilinear",
      batch_size=ds_batch_size,
  )

  return dataset

# **Build and Train Model Functions**

In [ ]:
#Function for normalizing images and labels
def normalize_img(image, label):
  """Noramlizes images: 'uint8 --> 'float32'."""
  return tf.cast(image, tf.float32) / 255., tf.cast(label, tf.int64)

In [ ]:
#Function that creates CNN Model Architecture that works for MNIST
def create_model_mnist():
  """
  model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10), 
  ])
    
  model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10), 
  ])
  """
  
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.AveragePooling2D(pool_size=(2,2), strides=2),
      tf.keras.layers.Conv2D(32, (5,5), activation='relu'),
      tf.keras.layers.Dropout(0.25),
      tf.keras.layers.AveragePooling2D(pool_size=(2,2), strides=2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(10, activation='softmax'),
  ])
 
  model.compile(
    optimizer = tf.keras.optimizers.Adam(0.001),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(), #from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
  )

  return model

In [ ]:
#Function that creates CNN Model Architecture that works for CIFAR10
def create_model_cifar10():

  """
  model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10),  
  ])

  """

  model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (4,4), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.Conv2D(64, (4,4), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(128, (4, 4), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.Conv2D(128, (4, 4), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Dropout(0.4),
    #tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(10, activation = 'softmax'),  
  ])

  """   
  model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),   
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.2),    
    tf.keras.layers.Dense(10, activation = 'softmax'),  
  ])
  """
  model.compile(
    optimizer = tf.keras.optimizers.Adam(0.001),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(), #from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
  )
  
  return model

In [ ]:
#Function for building CNN model, training it, and logging training metrics
#Takes train dataset, test dataset, log directory, directory type (clean, triggered), dataset directory, number of epochs, and percent of added images as parameters
#Returns history of model (metrics captured), log directory where metrics have been logged, and number of epochs used for training model
def build_train_model_and_log_metrics(train_ds, test_ds, log_dir, dir_type, ds_dir, ds_num_train_samples, num_epochs, added_percentage):

  AUTOTUNE = tf.data.AUTOTUNE

  train_ds = train_ds.map(normalize_img, num_parallel_calls=AUTOTUNE)                                                                
  train_ds = train_ds.cache().shuffle(ds_num_train_samples).prefetch(buffer_size=AUTOTUNE)

  test_ds = test_ds.map(normalize_img, num_parallel_calls=AUTOTUNE)
  test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

  logdir = log_dir.joinpath("{}_{}/{}_{}_{}".format(dir_type, ds_dir, dir_type, ds_dir, added_percentage))
  logdir.mkdir(parents=True)
  logdir.joinpath('training.csv').touch()
  csv_logger = CSVLogger(logdir.joinpath('training.csv'))

  #earlystopping_critera = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

  model = eval(f'create_model_{ds_dir}()')
  model.summary()

  #Creates custom callback function to be used for capturing per-class/label metrics at each epoch
  class Prediction_and_TrueLabel_Callback(tf.keras.callbacks.Callback):
    def __init__(self, model, dataset):

      self.model = model
      self.dataset = dataset
  
    def on_epoch_end(self, epoch, logs={}):
    
      filename = "predictions_and_true_labels" 
    
      y_true, y_pred = get_actual_predicted_labels(self.model, self.dataset)    

      pd.DataFrame({'y_pred':y_pred}).assign(y_true=y_true).to_csv('{}_{}.csv'.format(filename, epoch))
      
      per_epoch_metrics_dir = logdir.joinpath("per_epoch_metrics/epoch_{}/".format(epoch))
      per_epoch_metrics_dir.mkdir(parents=True)

      my_file = "predictions_and_true_labels_{}.csv".format(epoch)
      to_file = per_epoch_metrics_dir.joinpath(my_file)

      #Checks if file already exist in destination directory. If not, moves file in corresponding directory. If it exist, prints message notifying user.    
      if not (to_file.is_file()):
        shutil.move(my_file, to_file)
      else:
        print("File already exist in destination:" + str(to_file))
      
  history = model.fit(
      train_ds,
      validation_data=test_ds,
      epochs = num_epochs,
      callbacks=[csv_logger, Prediction_and_TrueLabel_Callback(model, train_ds)] #earlystopping_critera],
  )

  return history, logdir

In [ ]:
#Function for getting true labels and predicted labels for given dataset and returns them as two lists (actual_labels, predictions)
def get_actual_predicted_labels(model, dataset): 
  """
  Create a list of actual ground truth values and the predictions from the model.

  Args:
    dataset: An iterable data structure, such as a TensorFlow Dataset, with features and labels.

  Return:
    Ground truth and predicted values for a particular dataset.
  """
  predictions = []
  actual_labels = []

  for x, y in dataset:
    predictions.append(np.argmax(model(x), axis=-1))
    actual_labels.append(y.numpy())

  predictions = np.concatenate(predictions, axis=0)
  actual_labels = np.concatenate(actual_labels, axis=0)

  return actual_labels, predictions


# **Plot Functions**

In [ ]:
#Function for plotting confusion matrix and saves figure for current epoch in its corresponding directory
#Takes list of actual labels, predicted labels, label / class names of dataset, current epoch, dataset type (clean, triggered), and current epoch directory for logging per class metrics as parameters
def plot_confusion_matrix(actual, predicted, labels, epoch, ds_type, per_epoch_metrics_dir):
  cm = tf.math.confusion_matrix(actual, predicted)
  ax = sns.heatmap(cm, annot=True, fmt='g')
  sns.set(rc={'figure.figsize':(12, 12)})
  sns.set(font_scale=1.4)
  ax.set_title('Confusion matrix of Label Recognition for ' + ds_type)
  ax.set_xlabel('Predicted Label')
  ax.set_ylabel('Actual Label')
  plt.xticks(rotation=90)
  plt.yticks(rotation=0) 
  ax.xaxis.set_ticklabels(labels)
  ax.yaxis.set_ticklabels(labels)
  plt.savefig('confusion_matrix_epoch_{}.png'.format(epoch))
  plt.close()

  my_file = "confusion_matrix_epoch_{}.png".format(epoch)
  to_file = per_epoch_metrics_dir.joinpath(my_file)

  #Checks if file already exist in destination directory. If not, moves file in corresponding directory. If it exist, prints message notifying user.    
  if not (to_file.is_file()):
    shutil.move(my_file, to_file)
  else:
    print("File already exist in destination:" + str(to_file))

In [ ]:
#Function for plotting loss vs. epoch, accuracy vs. epoch. Saves figures in specific dataset type (clean, triggered) directory
#Takes history of model, dataset type, and log directory where all metrics are stored as parameters
def plot_history(history, ds_type, log_dir):
  """
    Plotting training and validation learning curves.

    Args:
      history: model history with all the metric measures
  """
  fig, (ax1, ax2) = plt.subplots(2)
  fig.set_size_inches(20, 14)

  # Plot loss
  ax1.set_title('Loss')
  ax1.plot(history.history['loss'], label = 'train')
  ax1.plot(history.history['val_loss'], label = 'test')
  ax1.set_ylabel('Loss')

  # Determine upper bound of y-axis
  max_loss = max(history.history['loss'] + history.history['val_loss'])

  ax1.set_ylim([0, np.ceil(max_loss)])
  ax1.set_xlabel('Epoch')
  ax1.legend(['Train', 'Validation']) 

  # Plot accuracy
  ax2.set_title('Accuracy')
  ax2.plot(history.history['sparse_categorical_accuracy'],  label = 'train')
  ax2.plot(history.history['val_sparse_categorical_accuracy'], label = 'test')
  ax2.set_ylabel('Accuracy')
  ax2.set_ylim([0, 1])
  ax2.set_xlabel('Epoch')
  ax2.legend(['Train', 'Validation'])

  plt.savefig('Loss_and_Accuracy_for_{}_Dataset.png'.format(ds_type))
  plt.close()

  my_file = 'Loss_and_Accuracy_for_{}_Dataset.png'.format(ds_type)
  to_file = log_dir.joinpath(my_file)

  #Checks if file already exist in destination directory. If not, moves file in corresponding directory. If it exist, prints message notifying user.    
  if not (to_file.is_file()):
    shutil.move(my_file, to_file)
  else:
    print("File already exist in destination:" + str(to_file))

In [ ]:
#Function for plotting clean and triggered loss vs. epoch on same plot. Repeats this for accuracy vs. epoch. Saves and stores figures in log directory.
#Takes histories and log directory as parameters
def plot_histories(history1, history2, history3, history4, history5, log_dir, ds_type):
  """
    Plotting training and validation learning curves.

    Args:
      history: model history with all the metric measures
  """
  fig, (ax1, ax2) = plt.subplots(2)
  fig.set_size_inches(20, 16)

  # Plot loss
  ax1.set_title('Loss')
  
  ax1.plot(history1.history['loss'], label = 'train')
  ax1.plot(history1.history['val_loss'], label = 'test')

  ax1.plot(history2.history['loss'], label = 'train')
  ax1.plot(history2.history['val_loss'], label = 'test')

  ax1.plot(history3.history['loss'], label = 'train')
  ax1.plot(history3.history['val_loss'], label = 'test')

  ax1.plot(history4.history['loss'], label = 'train')
  ax1.plot(history4.history['val_loss'], label = 'test')

  ax1.plot(history5.history['loss'], label = 'train')
  ax1.plot(history5.history['val_loss'], label = 'test')

  ax1.set_ylabel('Loss')

  # Determine upper bound of y-axis
  max_loss = max(history1.history['loss'] + history1.history['val_loss'] + history2.history['loss'] + history2.history['val_loss'] 
                 + history3.history['loss'] + history3.history['val_loss'] + history4.history['loss'] + history4.history['val_loss']
                 + history5.history['loss'] + history5.history['val_loss'])

  ax1.set_ylim([0, np.ceil(max_loss)])
  ax1.set_xlabel('Epoch')
  ax1.legend(['Clean Train', 'Clean Validation', 
              '{} 05 Train'.format(ds_type), '{} 05 Test'.format(ds_type), 
              '{} 10 Train'.format(ds_type), '{} 10 Test'.format(ds_type),
              '{} 15 Train'.format(ds_type), '{} 15 Test'.format(ds_type),
              '{} 20 Train'.format(ds_type), '{} 20 Test'.format(ds_type)]) 

  # Plot accuracy
  ax2.set_title('Accuracy')
  ax2.plot(history1.history['sparse_categorical_accuracy'],  label = 'train')
  ax2.plot(history1.history['val_sparse_categorical_accuracy'], label = 'test')
  
  ax2.plot(history2.history['sparse_categorical_accuracy'],  label = 'train')
  ax2.plot(history2.history['val_sparse_categorical_accuracy'], label = 'test')

  ax2.plot(history3.history['sparse_categorical_accuracy'],  label = 'train')
  ax2.plot(history3.history['val_sparse_categorical_accuracy'], label = 'test')

  ax2.plot(history4.history['sparse_categorical_accuracy'],  label = 'train')
  ax2.plot(history4.history['val_sparse_categorical_accuracy'], label = 'test')

  ax2.plot(history5.history['sparse_categorical_accuracy'],  label = 'train')
  ax2.plot(history5.history['val_sparse_categorical_accuracy'], label = 'test')

  ax2.set_ylabel('Accuracy')
  ax2.set_ylim([0, 1])
  ax2.set_xlabel('Epoch')
  ax2.legend(['Clean Train', 'Clean Validation', 
              '{} 05 Train'.format(ds_type), '{} 05 Test'.format(ds_type), 
              '{} 10 Train'.format(ds_type), '{} 10 Test'.format(ds_type),
              '{} 15 Train'.format(ds_type), '{} 15 Test'.format(ds_type),
              '{} 20 Train'.format(ds_type), '{} 20 Test'.format(ds_type)])
  
  plt.savefig('Loss_and_Accuracy_for_Original_Dataset_vs_{}_Datasets.png'.format(ds_type))
  plt.close()

  histories_dir = log_dir.joinpath('Training_Provenance/Captured_History_Plots/Overall_Dataset_Metric_Plots')
  histories_dir.mkdir(exist_ok=True, parents=True)

  my_file = 'Loss_and_Accuracy_for_Original_Dataset_vs_{}_Datasets.png'.format(ds_type)
  to_file = histories_dir.joinpath(my_file)

  #Checks if file already exist in destination directory. If not, moves file in corresponding directory. If it exist, prints message notifying user.    
  if not (to_file.is_file()):
    shutil.move(my_file, to_file)
  else:
    print("File already exist in destination:" + str(to_file))

In [ ]:
#Helper function to setup all per_class_metric_df for all epochs needed to plot all clean or triggered datasets along original dataset on same plot
#Takes per_epoch_dir, dir_name, filename as parameters
#Returns dataframe
def per_class_metric_df(epochs, per_epoch_metrics_dir):

  per_class_metric_df = pd.DataFrame()

  for x in range(epochs):

    dir_name = "epoch_{}".format(x) 
    filename = "per_class_metrics_per_epoch_{}.csv".format(x)

    tmp_path = per_epoch_metrics_dir.joinpath(dir_name).joinpath(filename)
    tmp_df = pd.read_csv(tmp_path)
    tmp_df['epoch'] = dir_name
    per_class_metric_df = pd.concat([per_class_metric_df, tmp_df])

  return per_class_metric_df

In [ ]:
#Helper function to create metric list from dataframes that were created
#Takes in substitute dataframe as parameter
#Returns metric list
def per_class_metric_list(sub_df, metric):
    
    per_class_metric = list()

    for idx_2, row in sub_df.iterrows():
      per_class_metric.append(row[metric])

    return per_class_metric

In [ ]:
#Function that plots per_class_metrics vs epoch for clean_augmented/triggered and original dataset on the same plot. Saves them to log directory
#Passes num of epochs, labels, log directory, original log directory, and clean/triggered log directories as parameters
#Returns original dataframe and clean_augmented/triggered dataframe with labels and per class metrics
def plot__per_class_metrics__vs__epoch(epochs, labels, log_dir, original_log_dir, log_dir_05, log_dir_10, log_dir_15, log_dir_20, ds_type):

  per_class_metric_plots_dir = log_dir.joinpath('Training_Provenance/Captured_History_Plots/Per_Class_Metric_Plots/Original_vs_{}_Per_Class_Metric_Plots/'.format(ds_type))
  per_class_metric_plots_dir.mkdir(exist_ok=True, parents = True)

  original_per_epoch_metrics_dir = original_log_dir.joinpath("per_epoch_metrics")
  
  per_epoch_metrics_05_dir = log_dir_05.joinpath("per_epoch_metrics")
  per_epoch_metrics_10_dir = log_dir_10.joinpath("per_epoch_metrics")
  per_epoch_metrics_15_dir = log_dir_15.joinpath("per_epoch_metrics")
  per_epoch_metrics_20_dir = log_dir_20.joinpath("per_epoch_metrics")

  original_df = per_class_metric_df(epochs, original_per_epoch_metrics_dir)
  
  df_05 = per_class_metric_df(epochs, per_epoch_metrics_05_dir)
  df_10 = per_class_metric_df(epochs, per_epoch_metrics_10_dir)
  df_15 = per_class_metric_df(epochs, per_epoch_metrics_15_dir)
  df_20 = per_class_metric_df(epochs, per_epoch_metrics_20_dir)

  per_class_metrics = list(original_df.columns)[1:-1]

  for label in original_df.Labels.unique():

    figure, axis = plt.subplots(1, 3)
    plt.subplots_adjust(wspace=0.4)

    figure.suptitle(f'Label: {label}', fontsize="xx-large")

    for idx, metric in enumerate(per_class_metrics):
      
      sub_df_original = original_df[original_df['Labels'] == label][[metric]]
      
      sub_df_05 = df_05[df_05['Labels'] == label][[metric]]
      sub_df_10 = df_10[df_10['Labels'] == label][[metric]]
      sub_df_15 = df_15[df_15['Labels'] == label][[metric]]
      sub_df_20 = df_20[df_10['Labels'] == label][[metric]]

      original_per_class_metric = per_class_metric_list(sub_df_original, metric)

      per_class_metric_05 = per_class_metric_list(sub_df_05, metric)
      per_class_metric_10 = per_class_metric_list(sub_df_10, metric)
      per_class_metric_15 = per_class_metric_list(sub_df_15, metric)
      per_class_metric_20 = per_class_metric_list(sub_df_20, metric)

      axis[idx].plot(list(range(epochs)), original_per_class_metric)
      axis[idx].plot(list(range(epochs)), per_class_metric_05)
      axis[idx].plot(list(range(epochs)), per_class_metric_10)
      axis[idx].plot(list(range(epochs)), per_class_metric_15)
      axis[idx].plot(list(range(epochs)), per_class_metric_20)

      axis[idx].set_title(metric)
      axis[idx].set_ylabel(metric)
      axis[idx].set_xlabel('epoch')
      axis[idx].legend(['Clean', '{}_05'.format(ds_type), '{}_10'.format(ds_type), '{}_15'.format(ds_type),'{}_20'.format(ds_type)])

    figure.set_size_inches(32, 12)

    plt.savefig('Per_Class_{}_Metric_Plots.png'.format(label))
    plt.close()

    my_file = 'Per_Class_{}_Metric_Plots.png'.format(label)
    to_file = per_class_metric_plots_dir.joinpath(my_file)

    #Checks if file already exist in destination directory. If not, moves file in corresponding directory. If it exist, prints message notifying user.    
    if not (to_file.is_file()):
      shutil.move(my_file, to_file)
    else:
      print("File already exist in destination:" + str(to_file))
  
  return original_df, df_05, df_10, df_15, df_20

# **Per-Class Functions**

In [ ]:
#Function for calculating precision, recall, and support from given confusion matrix
#Takes actual labels, predicted labels, and label / class names for given dataset as parameters
#Returns calculated precision, recall, and support for each label
def calculate_classification_metrics(y_actual, y_pred, labels):
  """
    Calculate the precision and recall of a classification model using the ground truth and
    predicted values. 

    Args:
      y_actual: Ground truth labels.
      y_pred: Predicted labels.
      labels: List of classification labels.

    Return:
      Precision, recall, support measures.
  """
  cm = tf.math.confusion_matrix(y_actual, y_pred)
  tp = np.diag(cm) # Diagonal represents true positives
  
  precision = dict()
  recall = dict()
  support = dict()

  for i in range(len(labels)):
  
    col = cm[:, i]
    fp = np.sum(col) - tp[i] # Sum of column minus true positive is false negative

    row = cm[i, :]
    fn = np.sum(row) - tp[i] # Sum of row minus true positive, is false negative
    
    precision[labels[i]] = tp[i] / (tp[i] + fp) # Precision 

    recall[labels[i]] = tp[i] / (tp[i] + fn) # Recall

    support[labels[i]] = np.sum(row)

  return precision, recall, support

In [ ]:
#Function for capturing per_class metrics at each epoch. Calls plot confusion matrix function and calculation function for precision, recall, and support.
#Writes precision, recall, support to csv file and saves that to per_class_metrics_per_epoch directory. 
#Takes number of epochs used for training model, log directory where other metrics are stored, label / class names for given dataset, and dataset type (clean, triggered) as parameters
def per_class_metrics_per_epoch(epochs, log_dir, labels, ds_type):

  for x in range(epochs):

    predictions = []
    actual_labels = []

    per_epoch_metrics_dir = log_dir.joinpath("per_epoch_metrics/epoch_{}/".format(x))
    predictions_and_true_labels = pd.read_csv(per_epoch_metrics_dir.joinpath("predictions_and_true_labels_{}.csv".format(x)))  

    for index, row in predictions_and_true_labels.iterrows():
      
      predictions.append(row['y_pred'])
      actual_labels.append(row['y_true'])

    plot_confusion_matrix(actual_labels, predictions, labels, x, ds_type, per_epoch_metrics_dir)
    precision, recall, support = calculate_classification_metrics(actual_labels, predictions, labels)

    merged_dict = defaultdict(list)
    dict_list = [precision, recall, support]

    for dict in dict_list:
      for k, v in dict.items():
        merged_dict[k].append(v)

    with open('per_class_metrics_per_epoch_{}.csv'.format(x), 'w') as outputFile:
      writer = csv.writer(outputFile)
      writer.writerow(['Labels', 'Precision', 'Recall', 'Support'])
      for key, value in merged_dict.items():
        writer.writerow([key, *value])
    
    outputFile.close()

    my_file = "per_class_metrics_per_epoch_{}.csv".format(x)
    to_file = per_epoch_metrics_dir.joinpath(my_file)

    #Checks if file already exist in destination directory. If not, moves file in corresponding directory. If it exist, prints message notifying user.    
    if not (to_file.is_file()):
      shutil.move(my_file, to_file)
    else:
      print("File already exist in destination:" + str(to_file)) 

# **Distance Formula Functions (Euclidean, Distance at Epoch etc.)**

In [ ]:
#Function for calculating distance between clean and triggered vectors (loss vs. epoch, accuracy vs. epoch). Saves values to csv file.
#Returns train_loss_dist_epoch, train_acc_dist_epoch, val_loss_dist_epoch, val_acc_dist_epoch to be used by multi_bar chart function later on
def calculate_distance_for_loss_and_acc(log_dir, log_dir1, log_dir2, run_num, added_percentage, ds_type):

    original_clean_ds_metrics = pd.read_csv(log_dir1.joinpath("training.csv"))
    modified_ds_metrics = pd.read_csv(log_dir2.joinpath("training.csv"))

    epoch = []
    
    original_train_loss = []
    original_train_acc = []
    original_val_loss = []
    original_val_acc = []

    modified_train_loss = []
    modified_train_acc = []
    modified_val_loss = []
    modified_val_acc = []

    for index, row in original_clean_ds_metrics.iterrows():

      epoch.append(int(row['epoch']))
      original_train_loss.append(float(row['loss']))
      original_train_acc.append(float(row['sparse_categorical_accuracy']))
      original_val_loss.append(float(row['val_loss']))
      original_val_acc.append(float(row['val_sparse_categorical_accuracy']))

    for index, row in modified_ds_metrics.iterrows():
      
      modified_train_loss.append(float(row['loss']))
      modified_train_acc.append(float(row['sparse_categorical_accuracy']))
      modified_val_loss.append(float(row['val_loss']))
      modified_val_acc.append(float(row['val_sparse_categorical_accuracy']))
   
    # Calculates euclidean distance for training/validation loss and accuracy
    train_loss_euclidean_dist = euclidean_distance(original_train_loss, modified_train_loss)
    train_acc_euclidean_dist = euclidean_distance(original_train_acc, modified_train_acc)

    val_loss_euclidean_dist = euclidean_distance(original_val_loss, modified_val_loss)
    val_acc_euclidean_dist = euclidean_distance(original_val_acc, modified_val_acc)

    # Calculates distance at epoch for training/validation loss and accuracy, the average, min, and max, and calls function to grab corresponding bar chart graph
    train_loss_dist_epoch  = distance_at_epoch(original_train_loss, modified_train_loss)
    train_loss_dist_epoch_avg, train_loss_dist_epoch_min, train_loss_dist_epoch_max = dist_at_epoch_avg_min_max(train_loss_dist_epoch) 
    get_bar_graph_dist_epochs(log_dir, ds_type, epoch, train_loss_dist_epoch, "Dist Across Epochs for {} {}%".format(ds_type, added_percentage), "Epochs", "Train Loss Distance", added_percentage)

    train_acc_dist_epoch = distance_at_epoch(original_train_acc, modified_train_acc)
    train_acc_dist_epoch_avg, train_acc_dist_epoch_min, train_acc_dist_epoch_max = dist_at_epoch_avg_min_max(train_acc_dist_epoch)
    get_bar_graph_dist_epochs(log_dir, ds_type, epoch, train_acc_dist_epoch, "Dist Across Epochs for {} {}%".format(ds_type, added_percentage), "Epochs", "Train Acc Distance", added_percentage)
    
    val_loss_dist_epoch = distance_at_epoch(original_val_loss, modified_val_loss)
    val_loss_dist_epoch_avg, val_loss_dist_epoch_min, val_loss_dist_epoch_max = dist_at_epoch_avg_min_max(val_loss_dist_epoch)
    get_bar_graph_dist_epochs(log_dir, ds_type, epoch, val_loss_dist_epoch, "Dist Across Epochs for {} {}%".format(ds_type, added_percentage), "Epochs", "Val Loss Distance", added_percentage)

    val_acc_dist_epoch = distance_at_epoch(original_val_acc, modified_val_acc)
    val_acc_dist_epoch_avg, val_acc_dist_epoch_min, val_acc_dist_epoch_max = dist_at_epoch_avg_min_max(val_acc_dist_epoch)
    get_bar_graph_dist_epochs(log_dir, ds_type, epoch, val_acc_dist_epoch, "Dist Across Epochs for {} {}%".format(ds_type, added_percentage), "Epochs", "Val Acc Distance", added_percentage)

    #Directory where Calculated Distance CSV logs will be stored
    dist_csv_dir = log_dir.joinpath('Training_Provenance/Calculated_Distance_CSV_Logs/Overall_Dataset')
    dist_csv_dir.mkdir(exist_ok=True, parents=True)

    #CSV file for Euclidean Distance
    my_file1 = 'euclidean_distance_between_{}_vs_Original_Loss_and_Acc_Vectors_.csv'.format(ds_type)
    to_file1 = dist_csv_dir.joinpath(my_file1)

    #CSV file for Distance at Epoch
    my_file2 = 'distance_at_epoch_between_{}_vs_Original_Loss_and_Acc_Vectors.csv'.format(ds_type)
    to_file2 = dist_csv_dir.joinpath(my_file2)

    #Checks if file already exist in destination directory. If not, moves file in corresponding directory. If it exist, prints message notifying user.    
    if not (to_file1.is_file()):

      with open(my_file1, 'w') as outputFile1:
        writer = csv.writer(outputFile1)
        writer.writerow(['Run','Train Loss Euclidean Dist', 'Train Acc Euclidean Dist', 'Val Loss Euclidean Dist', 'Val Acc Euclidean Dist', '{} Percentage'.format(ds_type)])
        writer.writerow([run_num, train_loss_euclidean_dist, train_acc_euclidean_dist, val_loss_euclidean_dist, val_acc_euclidean_dist, added_percentage])
         
      outputFile1.close()

      shutil.move(my_file1, to_file1)
    
    else:

      print("File already exist in destination:" + str(to_file1))
      print("\nAdding calculated Euclidean Distance for different {} Percentages...\n".format(ds_type))

      with open(to_file1, 'a') as outputFile1:
        writer = csv.writer(outputFile1)
        writer.writerow([run_num, train_loss_euclidean_dist, train_acc_euclidean_dist, val_loss_euclidean_dist, val_acc_euclidean_dist, added_percentage])
         
      outputFile1.close()

    #Checks if file already exist in destination directory. If not, moves file in corresponding directory. If it exist, prints message notifying user.    
    if not (to_file2.is_file()):

      with open(my_file2, 'w') as outputFile2:
        writer = csv.writer(outputFile2)
        writer.writerow(['Run', 'Epoch', 'Train Loss Dist', 'Train Acc Dist', 'Val Loss Dist', 'Val Acc Dist', '{} Percentage'.format(ds_type)])

        for x in epoch:
          
          writer.writerow([run_num, epoch[x], train_loss_dist_epoch[x], train_acc_dist_epoch[x], val_loss_dist_epoch[x], val_acc_dist_epoch[x], added_percentage])

        writer.writerow([''])
        writer.writerow(['Average Dist Across Epochs:'])
        writer.writerow(['', '', 'Train Loss Dist Avg', 'Train Acc Dist Avg', 'Val Loss Dist Avg', 'Val Acc Dist avg', '{} Percentage'.format(ds_type)])
        writer.writerow(['', '', train_loss_dist_epoch_avg, train_acc_dist_epoch_avg, val_loss_dist_epoch_avg, val_acc_dist_epoch_avg, added_percentage])

        writer.writerow([''])
        writer.writerow(['Minimum Dist Across Epochs:'])
        writer.writerow(['', '', 'Train Loss Dist Min', 'Train Acc Dist Min', 'Val Loss Dist Min', 'Val Acc Dist Min', '{} Percentage'.format(ds_type)])
        writer.writerow(['', '', train_loss_dist_epoch_min, train_acc_dist_epoch_min, val_loss_dist_epoch_min, val_acc_dist_epoch_min, added_percentage])
        writer.writerow(['','Epoch:', get_epoch(epoch, train_loss_dist_epoch, train_loss_dist_epoch_min), get_epoch(epoch, train_acc_dist_epoch, train_acc_dist_epoch_min), get_epoch(epoch, val_loss_dist_epoch, val_loss_dist_epoch_min), get_epoch(epoch, val_acc_dist_epoch, val_acc_dist_epoch_min)])  
                        
        writer.writerow([''])
        writer.writerow(['Maximum Dist Across Epochs:'])
        writer.writerow(['', '', 'Train Loss Dist Max', 'Train Acc Dist Max', 'Val Loss Dist Max', 'Val Acc Dist Max', '{} Percentage'.format(ds_type)])
        writer.writerow(['', '', train_loss_dist_epoch_min, train_acc_dist_epoch_min, val_loss_dist_epoch_min, val_acc_dist_epoch_min, added_percentage])
        writer.writerow(['','Epoch:', get_epoch(epoch, train_loss_dist_epoch, train_loss_dist_epoch_max), get_epoch(epoch, train_acc_dist_epoch, train_acc_dist_epoch_max), get_epoch(epoch, val_loss_dist_epoch, val_loss_dist_epoch_max), get_epoch(epoch, val_acc_dist_epoch, val_acc_dist_epoch_max)]) 
        writer.writerow(['']) 

      outputFile2.close()

      shutil.move(my_file2, to_file2)

    else:

      print("File already exist in destination:" + str(to_file2))
      print("\nAdding calculated Distance at Epoch for different {} Percentages...\n".format(ds_type))

      with open(to_file2, 'a') as outputFile2:
      
        writer = csv.writer(outputFile2)
        writer.writerow(['Run', 'Epoch', 'Train Loss Dist', 'Train Acc Dist', 'Val Loss Dist', 'Val Acc Dist', '{} Percentage'.format(ds_type)])

        for x in epoch:

          writer.writerow([run_num, epoch[x], train_loss_dist_epoch[x], train_acc_dist_epoch[x], val_loss_dist_epoch[x], val_acc_dist_epoch[x], added_percentage])

        writer.writerow([''])
        writer.writerow(['Average Dist Across Epochs:'])
        writer.writerow(['', '', 'Train Loss Dist Avg', 'Train Acc Dist Avg', 'Val Loss Dist Avg', 'Val Acc Dist avg', '{} Percentage'.format(ds_type)])
        writer.writerow(['', '', train_loss_dist_epoch_avg, train_acc_dist_epoch_avg, val_loss_dist_epoch_avg, val_acc_dist_epoch_avg, added_percentage])

        writer.writerow([''])
        writer.writerow(['Minimum Dist Across Epochs:'])
        writer.writerow(['', '', 'Train Loss Dist Min', 'Train Acc Dist Min', 'Val Loss Dist Min', 'Val Acc Dist Min', '{} Percentage'.format(ds_type)])
        writer.writerow(['', '', train_loss_dist_epoch_min, train_acc_dist_epoch_min, val_loss_dist_epoch_min, val_acc_dist_epoch_min, added_percentage])
        writer.writerow(['','Epoch:', get_epoch(epoch, train_loss_dist_epoch, train_loss_dist_epoch_min), get_epoch(epoch, train_acc_dist_epoch, train_acc_dist_epoch_min), get_epoch(epoch, val_loss_dist_epoch, val_loss_dist_epoch_min), get_epoch(epoch, val_acc_dist_epoch, val_acc_dist_epoch_min)])  
                        
        writer.writerow([''])
        writer.writerow(['Maximum Dist Across Epochs:'])
        writer.writerow(['', '', 'Train Loss Dist Max', 'Train Acc Dist Max', 'Val Loss Dist Max', 'Val Acc Dist Max', '{} Percentage'.format(ds_type)])
        writer.writerow(['', '', train_loss_dist_epoch_min, train_acc_dist_epoch_min, val_loss_dist_epoch_min, val_acc_dist_epoch_min, added_percentage])
        writer.writerow(['','Epoch:', get_epoch(epoch, train_loss_dist_epoch, train_loss_dist_epoch_max), get_epoch(epoch, train_acc_dist_epoch, train_acc_dist_epoch_max), get_epoch(epoch, val_loss_dist_epoch, val_loss_dist_epoch_max), get_epoch(epoch, val_acc_dist_epoch, val_acc_dist_epoch_max)])
        writer.writerow([' ']) 

      outputFile2.close()

    metric_dist_epoch_lst = [train_loss_dist_epoch, train_acc_dist_epoch, val_loss_dist_epoch, val_acc_dist_epoch]

    return metric_dist_epoch_lst


In [ ]:
def cal_generalization_gap(log_dir, log_dir1, run_num, added_percentage, ds_type):

    dataset_metrics = pd.read_csv(log_dir1.joinpath("training.csv"))
    
    epoch = []
        
    train_acc = []
    val_acc = []

    for index, row in dataset_metrics.iterrows():

      epoch.append(int(row['epoch']))
      train_acc.append(float(row['sparse_categorical_accuracy']))
      val_acc.append(float(row['val_sparse_categorical_accuracy']))

    generalization_gap = [abs(train_acc_i - val_acc_i) for train_acc_i, val_acc_i in zip(train_acc, val_acc)]

    #Directory where Calculated Distance CSV logs will be stored
    gen_gap_csv_dir = log_dir.joinpath('Training_Provenance/Generalization_Gap/CSV_Logs/{}'.format(ds_type))
    gen_gap_csv_dir.mkdir(exist_ok=True, parents=True)

    #CSV file for Generalization Gaps
    my_file1 = 'generalization_gap_{}_{}.csv'.format(ds_type, added_percentage)
    to_file1 = gen_gap_csv_dir.joinpath(my_file1)

    with open(my_file1, 'w') as outputFile1:
      writer = csv.writer(outputFile1)
      writer.writerow(['Run','Epoch', 'Generalization Gap ({} {}%)'.format(ds_type, added_percentage)])
  
      for x in epoch:
          
        writer.writerow([run_num, epoch[x], generalization_gap[x]])
         
    outputFile1.close()

    #Checks if file already exist in destination directory. If not, moves file in corresponding directory. If it exist, prints message notifying user.    
    if not (to_file1.is_file()):

      shutil.move(my_file1, to_file1)
    
    else:

      print("File already exist in destination:" + str(to_file1))

In [ ]:
#Function that defines euclidean distance formula.
#Takes two values as parameters and returns calculated distance
def euclidean_distance(a, b):
  return sqrt(sum((e1-e2)**2 for e1, e2 in zip(a,b)))

#Function that defines distance formula to calculate distances at each epoch
#Take two lists as parameters and returns list of calculated distances at each epoch
def distance_at_epoch(a, b):
  
  dist_at_epoch = [sqrt((a_i - b_i)**2) for a_i, b_i in zip(a, b)]
  
  return dist_at_epoch

#Function for calculating dist at epoch avg, min, and max
def dist_at_epoch_avg_min_max(lst):

  dist_epoch_avg = Average(lst)
  dist_epoch_min = min(lst)
  dist_epoch_max = max(lst)

  return dist_epoch_avg, dist_epoch_min, dist_epoch_max

#Function for calculating average of list
def Average(lst):
  return sum(lst) / len(lst)

#Function for returning given epoch based on if condition is met:
def get_epoch(epoch, lst, condition_val):

  for x in range(len(epoch)):

    if(lst[x] == condition_val):

      return epoch[x]
  

In [ ]:
#Function for calculating distance between clean and triggered vectors (per-class metric vs. epoch). Saves values to csv file.
def calculate_distance_for_per_class(run_num, labels, original_df, modified_df, log_dir, added_percentage, ds_type):

  euclidean_dist_precision = []
  euclidean_dist_recall = []
  euclidean_dist_support = []

  epoch = []
  precision_dist_at_epoch = []
  recall_dist_at_epoch = []
  support_dist_at_epoch = []

  for label in original_df.Labels.unique():

    epoch = getattr(original_df[original_df['Labels'] == label], 'epoch').tolist()
    
    original_precision = getattr(original_df[original_df['Labels'] == label], 'Precision').tolist()
    modified_precision = getattr(modified_df[modified_df['Labels'] == label], 'Precision').tolist()
    original_recall = getattr(original_df[original_df['Labels'] == label], 'Recall').tolist()
    modified_recall = getattr(modified_df[modified_df['Labels'] == label], 'Recall').tolist()
    original_support = getattr(original_df[original_df['Labels'] == label], 'Support').tolist()
    modified_support = getattr(modified_df[modified_df['Labels'] == label], 'Support').tolist()

    euclidean_dist_precision.append(euclidean_distance(original_precision, modified_precision))
    euclidean_dist_recall.append(euclidean_distance(original_recall, modified_recall))
    euclidean_dist_support.append(euclidean_distance(original_support, modified_support))

    precision_dist_at_epoch.append(distance_at_epoch(original_precision, modified_precision))
    recall_dist_at_epoch.append(distance_at_epoch(original_recall, modified_recall))
    support_dist_at_epoch.append(distance_at_epoch(original_support, modified_support))

  dist_csv_dir = log_dir.joinpath('Training_Provenance/Calculated_Distance_CSV_Logs/Per_Class')
  dist_csv_dir.mkdir(exist_ok=True, parents=True)

  my_file1 = 'euclidean_dist_per_class_metrics_Original_vs_{}.csv'.format(ds_type)
  to_file1 = dist_csv_dir.joinpath(my_file1)

  my_file2 = 'dist_at_epoch_per_class_metrics_Original_vs_{}.csv'.format(ds_type)
  to_file2 = dist_csv_dir.joinpath(my_file2)

  #Checks if file already exist in destination directory. If not, moves file in corresponding directory. If it exist, prints message notifying user.    
  if not (to_file1.is_file()):

    with open(my_file1, 'w') as outputFile1:
      writer = csv.writer(outputFile1)
      writer.writerow(['Run','Label','Precision Euclidean Dist', 'Recall Euclidean Dist', 'Support Euclidean Dist','{} Percentage'.format(ds_type)])

      for x in labels:
        writer.writerow([run_num, labels[x], euclidean_dist_precision[x], euclidean_dist_recall[x], euclidean_dist_support[x], added_percentage])
  
    outputFile1.close()

    shutil.move(my_file1, to_file1)

  else:
    print("File already exist in destination:" + str(to_file1))
    print("\nAdding calculated Euclidean Distance for different {} Percentages...\n".format(ds_type))


    with open(to_file1, 'a') as outputFile1:
      writer = csv.writer(outputFile1)

      for x in labels:
        writer.writerow([run_num, labels[x], euclidean_dist_precision[x], euclidean_dist_recall[x], euclidean_dist_support[x], added_percentage])
  
    outputFile1.close()  
  
  #Checks if file already exist in destination directory. If not, moves file in corresponding directory. If it exist, prints message notifying user.    
  if not (to_file2.is_file()):

    with open(my_file2, 'w') as outputFile2:
      writer = csv.writer(outputFile2)
      writer.writerow(['Run','Label', 'Epoch', 'Precision Dist at Epoch', 'Recall Dist at Epoch', 'Support Dist at Epoch','{} Percentage'.format(ds_type)])

      for x in labels:
      
        for y in range(len(epoch)):

          writer.writerow([run_num, labels[x], epoch[y], precision_dist_at_epoch[x][y], recall_dist_at_epoch[x][y], support_dist_at_epoch[x][y], added_percentage])
  
        precision_dist_epoch_avg, precision_dist_epoch_min, precision_dist_epoch_max = dist_at_epoch_avg_min_max(precision_dist_at_epoch[x])
        recall_dist_epoch_avg, recall_dist_epoch_min, recall_dist_epoch_max = dist_at_epoch_avg_min_max(recall_dist_at_epoch[x])
        support_dist_epoch_avg, support_dist_epoch_min, support_dist_epoch_max = dist_at_epoch_avg_min_max(support_dist_at_epoch[x])

        get_bar_graph_perclass_dist_epochs(log_dir, ds_type, x, epoch, precision_dist_at_epoch[x], "Dist Across Epochs for Label {} {} {}%".format(x, ds_type, added_percentage), "Epochs", "Precision Dist", added_percentage)
        get_bar_graph_perclass_dist_epochs(log_dir, ds_type, x, epoch, recall_dist_at_epoch[x], "Dist Across Epochs for Label {} {} {}%".format(x, ds_type, added_percentage), "Epochs", "Recall Dist", added_percentage)
        get_bar_graph_perclass_dist_epochs(log_dir, ds_type, x, epoch, support_dist_at_epoch[x], "Dist Across Epochs for Label {} {} {}%".format(x, ds_type, added_percentage), "Epochs", "Support Dist", added_percentage)
        
        writer.writerow([' '])
        writer.writerow(['Average Dist Across Epochs for Label {}:'.format(x)])
        writer.writerow(['', '', 'Precision Dist Avg', 'Recall Dist Avg', 'Support Avg', '{} Percentage'.format(ds_type)])
        writer.writerow(['', '', precision_dist_epoch_avg, recall_dist_epoch_avg, support_dist_epoch_avg,  added_percentage])

        writer.writerow([''])
        writer.writerow(['Minimum Dist Across Epochs for Label {}:'.format(x)])
        writer.writerow(['', '', 'Precision Dist Min', 'Recall Dist Min', 'Support Dist Min', '{} Percentage'.format(ds_type)])
        writer.writerow(['', '',precision_dist_epoch_min, recall_dist_epoch_min, support_dist_epoch_min, added_percentage])
        writer.writerow(['','Epoch:', get_epoch(epoch, precision_dist_at_epoch[x], precision_dist_epoch_min), get_epoch(epoch, recall_dist_at_epoch[x], recall_dist_epoch_min), get_epoch(epoch, support_dist_at_epoch[x], support_dist_epoch_min)])  
                        
        writer.writerow([''])
        writer.writerow(['Maximum Dist Across Epochs for Label {}:'.format(x)])
        writer.writerow(['', '', 'Precision Dist Max', 'Recall Dist Max', 'Support Dist Max', '{} Percentage'.format(ds_type)])
        writer.writerow(['', '',precision_dist_epoch_max, recall_dist_epoch_max, support_dist_epoch_max, added_percentage])
        writer.writerow(['','Epoch:', get_epoch(epoch, precision_dist_at_epoch[x], precision_dist_epoch_max), get_epoch(epoch, recall_dist_at_epoch[x], recall_dist_epoch_max), get_epoch(epoch, support_dist_at_epoch[x], support_dist_epoch_max)])  
        writer.writerow([' ']) 

    outputFile2.close()

    shutil.move(my_file2, to_file2)
    
  else:

    print("File already exist in destination:" + str(to_file2))
    print("\nAdding calculated Distance at Epoch for different {} Percentages...\n".format(ds_type))

    with open(to_file2, 'a') as outputFile2:
      writer = csv.writer(outputFile2)
      writer.writerow(['Run','Label', 'Epoch', 'Precision Dist at Epoch', 'Recall Dist at Epoch', 'Support Dist at Epoch','{} Percentage'.format(ds_type)])

      for x in labels:
      
        for y in range(len(epoch)):

          writer.writerow([run_num, labels[x], epoch[y], precision_dist_at_epoch[x][y], recall_dist_at_epoch[x][y], support_dist_at_epoch[x][y], added_percentage])
  
        precision_dist_epoch_avg, precision_dist_epoch_min, precision_dist_epoch_max = dist_at_epoch_avg_min_max(precision_dist_at_epoch[x])
        recall_dist_epoch_avg, recall_dist_epoch_min, recall_dist_epoch_max = dist_at_epoch_avg_min_max(recall_dist_at_epoch[x])
        support_dist_epoch_avg, support_dist_epoch_min, support_dist_epoch_max = dist_at_epoch_avg_min_max(support_dist_at_epoch[x])

        get_bar_graph_perclass_dist_epochs(log_dir, ds_type, x, epoch, precision_dist_at_epoch[x], "Dist Across Epochs for Label {} {} {}%".format(x, ds_type, added_percentage), "Epochs", "Precision Dist", added_percentage)
        get_bar_graph_perclass_dist_epochs(log_dir, ds_type, x, epoch, recall_dist_at_epoch[x], "Dist Across Epochs for Label {} {} {}%".format(x, ds_type, added_percentage), "Epochs", "Recall Dist", added_percentage)
        get_bar_graph_perclass_dist_epochs(log_dir, ds_type, x, epoch, support_dist_at_epoch[x], "Dist Across Epochs for Label {} {} {}%".format(x, ds_type, added_percentage), "Epochs", "Support Dist", added_percentage)

        writer.writerow([' '])
        writer.writerow(['Average Dist Across Epochs for Label {}:'.format(x)])
        writer.writerow(['', '', 'Precision Dist Avg', 'Recall Dist Avg', 'Support Avg', '{} Percentage'.format(ds_type)])
        writer.writerow(['', '', precision_dist_epoch_avg, recall_dist_epoch_avg, support_dist_epoch_avg,  added_percentage])

        writer.writerow([''])
        writer.writerow(['Minimum Dist Across Epochs for Label {}:'.format(x)])
        writer.writerow(['', '', 'Precision Dist Min', 'Recall Dist Min', 'Support Dist Min', '{} Percentage'.format(ds_type)])
        writer.writerow(['', '',precision_dist_epoch_min, recall_dist_epoch_min, support_dist_epoch_min, added_percentage])
        writer.writerow(['','Epoch:', get_epoch(epoch, precision_dist_at_epoch[x], precision_dist_epoch_min), get_epoch(epoch, recall_dist_at_epoch[x], recall_dist_epoch_min), get_epoch(epoch, support_dist_at_epoch[x], support_dist_epoch_min)])  
                        
        writer.writerow([''])
        writer.writerow(['Maximum Dist Across Epochs for Label {}:'.format(x)])
        writer.writerow(['', '', 'Precision Dist Max', 'Recall Dist Max', 'Support Dist Max', '{} Percentage'])
        writer.writerow(['', '',precision_dist_epoch_max, recall_dist_epoch_max, support_dist_epoch_max, added_percentage])
        writer.writerow(['','Epoch:', get_epoch(epoch, precision_dist_at_epoch[x], precision_dist_epoch_max), get_epoch(epoch, recall_dist_at_epoch[x], recall_dist_epoch_max), get_epoch(epoch, support_dist_at_epoch[x], support_dist_epoch_max)])  
        writer.writerow([' ']) 

    outputFile2.close()

  perClass_metric_dist_epoch_lst = [precision_dist_at_epoch, recall_dist_at_epoch, support_dist_at_epoch]

  return perClass_metric_dist_epoch_lst

# **Experimental Functions for executing standard functions for training ML model and for capturing training provenance**

In [ ]:
#Runs experiment on clean datasets and captures/logs training provenance
def run_clean_experiments(logdir, dataset_directory, dataset_labels, num_epochs, num_train_samples, cleanTrain_dir, cleanTest_dir, img_ds_height, img_ds_width, dataset_color_mode, dataset_batch_size, augmented_percentage):

  #calls function to get clean datasets to be using for training and testing
  print("\nGetting Clean {} Training and Test Dataset ({}% Augmented) ...\n".format(dataset_directory.upper(), augmented_percentage))
  clean_train_ds = get_dataset(cleanTrain_dir, img_ds_height, img_ds_width, dataset_color_mode, dataset_batch_size)
  clean_test_ds = get_dataset(cleanTest_dir, img_ds_height, img_ds_width, dataset_color_mode, dataset_batch_size)
          
  #calls function to build and train model on clean datasets as well as logs metrics captured during training and testing
  print("\nTraining CNN Model with Clean {} Datasets ({}% Augmented) and Logging Metrics ...\n".format(dataset_directory.upper(), augmented_percentage))
  history, clean_log_dir = build_train_model_and_log_metrics(clean_train_ds, clean_test_ds, logdir, "clean", dataset_directory, num_train_samples, num_epochs, augmented_percentage)

  #calls function to plot history loss and accuracy for clean dataset.
  print("\nSaving Loss and Accuracy Plot for Clean {} Dataset ({}% Augmented) ...\n".format(dataset_directory.upper(), augmented_percentage))
  plot_history(history, "Clean_{}".format(augmented_percentage), clean_log_dir)

  #calls function to generate confusion matrix plot and log precision, recall, support per class at every epoch
  print("\nSaving Confusion Matrix Plot and Logging Metrics per Epoch for Clean {} Train Dataset ({}% Augmented) ...\n".format(dataset_directory.upper(), augmented_percentage))
  per_class_metrics_per_epoch(num_epochs, clean_log_dir, dataset_labels, "clean_train_{}".format(augmented_percentage))

  return  history, clean_log_dir


In [ ]:
#Runs experiment on triggered datasets and captures/logs training provenance
def run_triggered_experiments(logdir, dataset_directory, dataset_labels, num_epochs, num_train_samples, triggered_percentage, triggeredTrain_dir, triggeredTest_dir, img_ds_height, img_ds_width, dataset_color_mode, dataset_batch_size):

  #calls function to get triggered datasets to be using for training and testing          
  print("\nGetting Triggered {} Training and Test Dataset ({}% Triggered)...\n".format(dataset_directory.upper(), triggered_percentage))
  triggered_train_ds = get_dataset(triggeredTrain_dir, img_ds_height, img_ds_width, dataset_color_mode, dataset_batch_size)
  triggered_test_ds = get_dataset(triggeredTest_dir, img_ds_height, img_ds_width, dataset_color_mode, dataset_batch_size)

  #calls function to build and train model on triggered datasets as well as logs metrics captured during training and testing
  print("\nTraining Model with Triggered {} Datasets ({}% Triggered) and Logging Metrics ...\n".format(dataset_directory.upper(), triggered_percentage))
  history, triggered_log_dir = build_train_model_and_log_metrics(triggered_train_ds, triggered_test_ds, logdir, "triggered", dataset_directory, num_train_samples, num_epochs, triggered_percentage)

  #calls function to plot history loss and accuracy for triggered dataset.
  print("\nSaving Loss and Accuracy Plot for Triggered {} Dataset ({}% Triggered) ...\n".format(dataset_directory.upper(), triggered_percentage))
  plot_history(history, "Triggered_{}".format(triggered_percentage), triggered_log_dir)

  #calls function to generate confusion matrix plot and log precision, recall, support per class at every epoch
  print("\nSaving Confusion Matrix Plot and Logging Metrics per Epoch for Triggered {} Train Dataset ({}% Triggered) ...\n".format(dataset_directory.upper(), triggered_percentage))
  per_class_metrics_per_epoch(num_epochs, triggered_log_dir, dataset_labels, "triggered_{}_train".format(triggered_percentage))

  return history, triggered_log_dir

# **Bar Graph Functions**



In [ ]:
def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        plt.text(rect.get_x() + rect.get_width()/2., 1.05*height, '%.4f' % float(height), fontsize='small', rotation=90, ha='center', va='bottom')


In [ ]:
#Function that gets bar graph for Euclidean Distance of Loss and Acc Vectors vs. Runs
def get_multi_bargraph_euclidean_dist_runs_overall_ds(dataset_directory, clean_euclidean_dist_df, triggered_euclidean_dist_df, percentage):

  metrics = ['Train Loss Euclidean Dist', 'Train Acc Euclidean Dist', 'Val Loss Euclidean Dist', 'Val Acc Euclidean Dist']

  lst1 = []
  lst2 = []
  lst3 = []

  for metric in metrics:

    bar_graph_dir = dir.joinpath("logs/scalars/Training_Provenance_Runs/{}/Bar_Graphs_Euclidean_Dist/Overall_Dataset/{}".format(dataset_directory, metric))
    bar_graph_dir.mkdir(exist_ok=True, parents = True)

    lst1 = getattr(clean_euclidean_dist_df[clean_euclidean_dist_df['Clean Percentage'] == percentage], 'Run').tolist()
    lst2 = getattr(clean_euclidean_dist_df[clean_euclidean_dist_df['Clean Percentage'] == percentage], metric).tolist()
    lst3 = getattr(triggered_euclidean_dist_df[triggered_euclidean_dist_df['Triggered Percentage'] == percentage], metric).tolist()

    X = lst1
    Y = lst2
    Z = lst3
   
    X_axis = np.arange(len(X))

    bar1 = plt.bar(X_axis - 0.2, Y, 0.2, label = 'Clean_{}'.format(percentage))
    autolabel(bar1)
    bar2 = plt.bar(X_axis + 0.2, Z, 0.2, label = 'Triggered_{}'.format(percentage))
    autolabel(bar2)


    plt.xticks(X_axis, X)
    plt.xlabel('Runs')
    plt.ylabel(metric)
    title = metric + " Across Runs ({} %)".format(percentage)
    #plt.title(title)
    plt.legend()

    fig_filename = metric + ' {}.png'.format(percentage)
    plt.savefig(fig_filename)
    plt.close()

    my_file = fig_filename
    to_file = bar_graph_dir.joinpath(my_file)

    #Checks if file already exist in destination directory. If not, moves file in corresponding directory. If it exist, prints message notifying user.    
    if not (to_file.is_file()):
      shutil.move(my_file, to_file)
    else:
      print("File already existed in destination:" + str(to_file))
      print("Overwritting File")
      shutil.move(my_file, to_file)


In [ ]:
#Function that gets bar graph for Euclidean Distance of Loss and Acc Vectors vs. Runs
def get_multi_bargraph_euclidean_dist_runs_perclass(dataset_directory, clean_euclidean_dist_df, triggered_euclidean_dist_df, percentage):

  metrics = ['Precision Euclidean Dist', 'Recall Euclidean Dist', 'Support Euclidean Dist']

  lst1 = []
  lst2 = []
  lst3 = []

  tmp_df1 = pd.DataFrame ()
  tmp_df2 = pd.DataFrame()

  for class_label in clean_euclidean_dist_df['Label'].unique():

    tmp_df1 = clean_euclidean_dist_df[clean_euclidean_dist_df['Label'] == class_label]
    tmp_df2 = triggered_euclidean_dist_df[triggered_euclidean_dist_df['Label'] == class_label]

    for metric in metrics:

      bar_graph_per_class = dir.joinpath("logs/scalars/Training_Provenance_Runs/{}/Bar_Graphs_Euclidean_Dist/Per_Class/{}/{}".format(dataset_directory, metric, class_label))
      bar_graph_per_class.mkdir(exist_ok=True, parents = True)

      lst1 = getattr(tmp_df1[tmp_df1['Clean Percentage'] == percentage], 'Run').tolist()
      lst2 = getattr(tmp_df1[tmp_df1['Clean Percentage'] == percentage], metric).tolist()
      lst3 = getattr(tmp_df2[tmp_df2['Triggered Percentage'] == percentage], metric).tolist()

      X = lst1
      Y = lst2
      Z = lst3
   
      X_axis = np.arange(len(X))

      bar1 = plt.bar(X_axis - 0.2, Y, 0.2, label = 'Clean_{}'.format(percentage))
      autolabel(bar1)
      bar2 = plt.bar(X_axis + 0.2, Z, 0.2, label = 'Triggered_{}'.format(percentage))
      autolabel(bar2)

      plt.xticks(X_axis, X)
      plt.xlabel('Runs')
      y_Label = metric + " (Label {})".format(class_label)
      plt.ylabel(y_Label)
      #title = metric + " Across Runs Label {} ({} %)".format(class_label, percentage)
      #plt.title(title)
      plt.legend()

      fig_filename = metric + 'for Label {} Percentage {}.png'.format(class_label, percentage)
      plt.savefig(fig_filename)
      plt.close()

      my_file = fig_filename
      to_file = bar_graph_per_class.joinpath(my_file)

      #Checks if file already exist in destination directory. If not, moves file in corresponding directory. If it exist, prints message notifying user.    
      if not (to_file.is_file()):
        shutil.move(my_file, to_file)
      else:
        print("File already exist in destination:" + str(to_file))
        print("Overwritting File")
        shutil.move(my_file, to_file)


In [ ]:
#Function for generating single bar graph
def generate_bar_graph(directory, lst1, lst2, plt_title, x_label, y_label):

  fig = plt.figure()
  ax = fig.add_axes([0.1, 0.1, 0.9, 0.9])
  bar1 = ax.bar(lst1,lst2)
  autolabel(bar1)
  plt.title(plt_title)
  plt.xlabel(x_label)
  plt.ylabel(y_label)
  fig_filename = y_label + ' ' + plt_title + '.png'
  plt.savefig(fig_filename)
  plt.close()

  my_file = fig_filename
  to_file = directory.joinpath(my_file)

  #Checks if file already exist in destination directory. If not, moves file in corresponding directory. If it exist, prints message notifying user.    
  if not (to_file.is_file()):
    shutil.move(my_file, to_file)
  else:
    print("File already exist in destination:" + str(to_file))

In [ ]:
#Function for generating multiple bar charts on same graph
def generate_multi_bar_graph(directory, lst1, lst2, lst3, plt_title, x_label, y_label, Y_Label, Z_Label):

  X = lst1
  Y = lst2
  Z = lst3
  
  X_axis = np.arange(len(X))
  
  bar1 = plt.bar(X_axis - 0.2, Y, 0.4, label = Y_Label)
  autolabel(bar1)
  bar2 = plt.bar(X_axis + 0.2, Z, 0.4, label = Z_Label)
  autolabel(bar2)

  plt.xticks(X_axis, X)
  plt.xlabel(x_label)
  plt.ylabel(y_label)
  #plt.title(plt_title)
  plt.legend()

  fig_filename = plt_title + '.png'
  plt.savefig(fig_filename)
  plt.close()

  my_file = fig_filename
  to_file = directory.joinpath(my_file)

  #Checks if file already exist in destination directory. If not, moves file in corresponding directory. If it exist, prints message notifying user.    
  if not (to_file.is_file()):
    shutil.move(my_file, to_file)
  else:
    print("File already exist in destination:" + str(to_file))

In [ ]:
#Function for generating multiple bar charts on same graph
def generate_multi_bar_graph_gen_gap(directory, added_percentage):

  #Directory where Calculated Distance CSV logs are stored
  gen_gap_csv_dir = directory.joinpath('Training_Provenance/Generalization_Gap/CSV_Logs/')
  gen_gap_csv_dir.mkdir(exist_ok=True, parents=True)

  gen_gap_bargraph_dir = directory.joinpath('Training_Provenance/Generalization_Gap/Bar_Graphs/')
  gen_gap_bargraph_dir.mkdir(exist_ok=True, parents=True)

  #Gen Gap Dataframes
  original_DF = pd.read_csv(gen_gap_csv_dir.joinpath('Original/generalization_gap_Original_0.csv'))
  clean_DF = pd.read_csv(gen_gap_csv_dir.joinpath('Clean/generalization_gap_Clean_{}.csv'.format(added_percentage)))
  triggered_DF = pd.read_csv(gen_gap_csv_dir.joinpath('Triggered/generalization_gap_Triggered_{}.csv'.format(added_percentage)))

  lst1 = original_DF['Epoch'].tolist()
  lst2 = original_DF['Generalization Gap (Original 0%)'].tolist()
  lst3 = clean_DF['Generalization Gap (Clean {}%)'.format(added_percentage)].tolist()
  lst4 = triggered_DF['Generalization Gap (Triggered {}%)'.format(added_percentage)].tolist()

  X = lst1
  Y = lst2
  Z1 = lst3
  Z2 = lst4
 
  X_axis = np.arange(len(X))
  
  bar1 = plt.bar(X_axis - 0.2, Y, 0.2, label = 'Original')
  bar2 = plt.bar(X_axis, Z1, 0.2, label = 'Clean_{}'.format(added_percentage))
  bar3 = plt.bar(X_axis + 0.2, Z2, 0.2, label = 'Triggered_{}'.format(added_percentage))
  autolabel(bar1) 
  autolabel(bar2) 
  autolabel(bar3)

  plt.xticks(X_axis, X)
  plt.xlabel('Epochs')
  plt.ylabel('Generalization Gap')
  plt.title("Generalization_Gap_{}".format(added_percentage))
  plt.legend()

  fig_filename = 'Generalization_Gap_{}.png'.format(added_percentage)
  plt.savefig(fig_filename)
  plt.close()

  my_file = fig_filename
  to_file = gen_gap_bargraph_dir.joinpath(my_file)

  #Checks if file already exist in destination directory. If not, moves file in corresponding directory. If it exist, prints message notifying user.    
  if not (to_file.is_file()):
    shutil.move(my_file, to_file)
  else:
    print("File already exist in destination:" + str(to_file))

In [ ]:
#Function for generating multiple bar charts on same graph
def generate_multi_bar_graph_training_trajectory_overall_ds(directory, ds_dir, added_percentage):

  #Directory where Calculated Distance CSV logs are stored
  original_training_csv_dir = directory.joinpath("clean_{}/clean_{}_0".format(ds_dir, ds_dir))
  clean_training_csv_dir = directory.joinpath("clean_{}/clean_{}_{}".format(ds_dir, ds_dir, added_percentage))
  triggered_training_csv_dir = directory.joinpath("triggered_{}/triggered_{}_{}".format(ds_dir, ds_dir, added_percentage))

  #Training Trajectory Dataframes
  original_DF = pd.read_csv(original_training_csv_dir.joinpath('training.csv'))
  clean_DF = pd.read_csv(clean_training_csv_dir.joinpath('training.csv'))
  triggered_DF = pd.read_csv(triggered_training_csv_dir.joinpath('training.csv'))

  overall_ds_metrics = ['loss', 'sparse_categorical_accuracy', 'val_loss', 'val_sparse_categorical_accuracy']

  lst1 = []
  lst2 = []
  lst3 = []
  lst4 = []

  for metric in overall_ds_metrics:

    #Directory where Training Trajectory Bargraphs are stored.
    training_trajectory_bargraph_dir = directory.joinpath('Training_Provenance/Training_Trajectory/Bar_Graphs/Overall_Dataset_Metric/{}'.format(metric))
    training_trajectory_bargraph_dir.mkdir(exist_ok=True, parents=True)

    lst1 = original_DF['epoch'].tolist()
    lst2 = original_DF[metric].tolist()
    lst3 = clean_DF[metric].tolist()
    lst4 = triggered_DF[metric].tolist()

    X = lst1
    Y = lst2
    Z1 = lst3
    Z2 = lst4
 
    X_axis = np.arange(len(X))
  
    bar1 = plt.bar(X_axis - 0.2, Y, 0.2, label = 'Original')
    bar2 = plt.bar(X_axis, Z1, 0.2, label = 'Clean_{}'.format(added_percentage))
    bar3 = plt.bar(X_axis + 0.2, Z2, 0.2, label = 'Triggered_{}'.format(added_percentage))
    autolabel(bar1)
    autolabel(bar2)
    autolabel(bar3)

    plt.xticks(X_axis, X)
    plt.xlabel('Epochs')
    plt.ylabel(metric)
    #plt.title("{}_trajectory_{}".format(metric, added_percentage))
    plt.legend()

    fig_filename = '{}_trajectory_{}.png'.format(metric, added_percentage)
    plt.savefig(fig_filename)
    plt.close()

    my_file = fig_filename
    to_file = training_trajectory_bargraph_dir.joinpath(my_file)

    #Checks if file already exist in destination directory. If not, moves file in corresponding directory. If it exist, prints message notifying user.    
    if not (to_file.is_file()):
      shutil.move(my_file, to_file)
    else:
      print("File already exist in destination:" + str(to_file))

In [ ]:
#Function for generating multiple bar charts on same graph
def generate_multi_bar_graph_training_trajectory_perclass(directory, ds_dir, original_df, clean_df, triggered_df, added_percentage):

  tmplst = []
  lst1 = []
  lst2 = []
  lst3 = []
  lst4 = []

  perclass_metrics = ['Precision', 'Recall', 'Support']

  for class_label in original_df.Labels.unique():
    
    for metric in perclass_metrics:

      #Directory where Training Trajectory Bargraphs are stored.
      training_trajectory_bargraph_dir = directory.joinpath('Training_Provenance/Training_Trajectory/Bar_Graphs/Per_Class_Metrics/{}/Label_{}'.format(metric, class_label))
      training_trajectory_bargraph_dir.mkdir(exist_ok=True, parents=True)

      tmplst = getattr(original_df[original_df['Labels'] == class_label], 'epoch').tolist()
      lst1 = [s.strip('epoch_') for s in tmplst]
      lst2 = getattr(original_df[original_df['Labels'] == class_label], metric).tolist()
      lst3 = getattr(clean_df[clean_df['Labels'] == class_label], metric).tolist()
      lst4 = getattr(triggered_df[triggered_df['Labels'] == class_label], metric).tolist()
      
      X = lst1
      Y = lst2
      Z1 = lst3
      Z2 = lst4
 
      X_axis = np.arange(len(X))
  
      bar1 = plt.bar(X_axis - 0.2, Y, 0.2, label = 'Original')
      bar2 = plt.bar(X_axis, Z1, 0.2, label = 'Clean_{}'.format(added_percentage))
      bar3 = plt.bar(X_axis + 0.2, Z2, 0.2, label = 'Triggered_{}'.format(added_percentage))
      autolabel(bar1)
      autolabel(bar2)
      autolabel(bar3)


      plt.xticks(X_axis, X)
      plt.xlabel('Epochs')
      y_Label = metric + " (Label {})".format(class_label)
      plt.ylabel(y_Label)
      #plt.title("{}_Trajectory_Label_{}_Percentage_{}".format(metric, class_label, added_percentage))
      plt.legend()

      fig_filename = '{}_Trajectory_Label_{}_Percentage_{}.png'.format(metric, class_label, added_percentage)
      plt.savefig(fig_filename)
      plt.close()

      my_file = fig_filename
      to_file = training_trajectory_bargraph_dir.joinpath(my_file)

      #Checks if file already exist in destination directory. If not, moves file in corresponding directory. If it exist, prints message notifying user.    
      if not (to_file.is_file()):
        shutil.move(my_file, to_file)
      else:
        print("File already exist in destination:" + str(to_file))

In [ ]:
#Function that gets single bar graph for distance at epoch for overall dataset metrics
def get_bar_graph_dist_epochs(directory, ds_type, lst1, lst2, plt_title, x_label, y_label, added_percentage):

  print("\nGenerating and Saving Bar Chart Graph that displays {} Across Epochs for {} ({}%) Dataset vs. Original Dataset ...\n".format(y_label, ds_type, added_percentage))
  
  bar_graph_dir = directory.joinpath("Training_Provenance/Bar_Graphs/Bar_Graph_Epochs_Overall_Dataset/Single_Bar_Chart/{}_{}".format(y_label, ds_type))
  bar_graph_dir.mkdir(exist_ok=True, parents = True)

  generate_bar_graph(bar_graph_dir, lst1, lst2, plt_title, x_label, y_label)
  

In [ ]:
#Function that gets single bar graph for distance at epoch for per_class metric distances
def get_bar_graph_perclass_dist_epochs(directory, ds_type, label, lst1, lst2, plt_title, x_label, y_label, added_percentage):

  print("\nGenerating and Saving Bar Chart Graph that displays {} Across Epochs for Label {} from {} ({}%) Dataset vs. Original Dataset ...\n".format(y_label, label, ds_type, added_percentage))
  
  bar_graph_per_class = directory.joinpath("Training_Provenance/Bar_Graphs/Bar_Graph_Per_Class_Epochs/Single_Bar_Chart/{}_{}/label_{}".format(y_label, ds_type, label))
  bar_graph_per_class.mkdir(exist_ok=True, parents = True)

  generate_bar_graph(bar_graph_per_class, lst1, lst2, plt_title, x_label, y_label)


In [ ]:
def get_multi_bar_graph_epochs(directory, num_epochs, lst1, lst2, percentage):

  multi_bar_graph_dir = directory.joinpath("Training_Provenance/Bar_Graphs/Bar_Graph_Epochs_Overall_Dataset/Multi_Bar_Chart/{}".format(percentage))
  multi_bar_graph_dir.mkdir(exist_ok=True, parents = True)  

  metrics_lst = ['Train Loss Dist', 'Train Acc Dist', 'Val Loss Dist', 'Val Acc Dist',]

  lst_epoch = []

  for epoch in range(num_epochs):

    lst_epoch.append(epoch)

  for metric in range(len(metrics_lst)):
    
    generate_multi_bar_graph(multi_bar_graph_dir, lst_epoch, lst1[metric], lst2[metric], '{} vs. Epochs (Clean & Triggered {}%)'.format(metrics_lst[metric], percentage), 'Epochs', metrics_lst[metric], 'Clean {}'.format(percentage), 'Triggered {}'.format(percentage))


In [ ]:
def get_multi_bar_graph_perclass_dist_epochs(directory, labels, num_epochs, clean_perclass_metric_dist_at_epoch, triggered_perclass_metric_dist_at_epoch, percentage):

  metrics_lst = ['Precision Distance', 'Recall Distance', 'Support Distance']

  lst_epoch = []

  for epoch in range(num_epochs):

    lst_epoch.append(epoch)
  
  for metric in range(len(metrics_lst)):

    for x in range(len(labels)):

      multi_bar_graph_per_class = directory.joinpath("Training_Provenance/Bar_Graphs/Bar_Graph_Per_Class_Epochs/Multi_Bar_Chart/{}/label_{}".format(metrics_lst[metric], x))
      multi_bar_graph_per_class.mkdir(exist_ok=True, parents = True)

      generate_multi_bar_graph(multi_bar_graph_per_class, lst_epoch, clean_perclass_metric_dist_at_epoch[metric][x], triggered_perclass_metric_dist_at_epoch[metric][x], '{} vs. Epochs Label {} (Clean & Triggered {}%)'.format(metrics_lst[metric], str(x), percentage), 'Epochs', metrics_lst[metric], 'Clean {}'.format(percentage), 'Triggered {}'.format(percentage))


# **Training Provenance Detection Function**

In [ ]:
#Function that calls necessary functions for plotting clean and triggered training provenance on same plot.
#Also calls function for calculating euclidean distance between clean and triggered vectors 
def training_prov_detection(dataset_directory, dataset_labels, epochs, logdir, original_logdir, 
                            clean_logdir_05, clean_logdir_10, clean_logdir_15, clean_logdir_20, 
                            triggered_logdir_05, triggered_logdir_10, triggered_logdir_15, triggered_logdir_20,
                            history1, clean_history2, clean_history3, clean_history4, clean_history5, 
                            triggered_history6, triggered_history7, triggered_history8, triggered_history9, 
                            count):    

    #calls function to plot history loss and accuracy for clean vs. original dataset.
    print("\nSaving Loss and Accuracy Plot for {} Clean Datasets vs. Original Dataset ...\n".format(dataset_directory.upper()))
    plot_histories(history1, clean_history2, clean_history3, clean_history4, clean_history5, logdir, 'Clean')

    #calls function to calculate distance for loss and accuracy vectors for clean vs. original dataset
    print("\nCalculating Distance between Loss and Accuracy Vectors for {} Clean Datasets vs. Original Dataset ...\n".format(dataset_directory.upper()))
    clean_metric_dist_epoch_lst_05 = calculate_distance_for_loss_and_acc(logdir, original_logdir, clean_logdir_05, count, '5', 'Clean')
    clean_metric_dist_epoch_lst_10 = calculate_distance_for_loss_and_acc(logdir, original_logdir, clean_logdir_10, count, '10', 'Clean')
    clean_metric_dist_epoch_lst_15 = calculate_distance_for_loss_and_acc(logdir, original_logdir, clean_logdir_15, count, '15', 'Clean')
    clean_metric_dist_epoch_lst_20 = calculate_distance_for_loss_and_acc(logdir, original_logdir, clean_logdir_20, count, '20', 'Clean')

    #calls function that generates and saves per class metrics plots for clean vs. original dataset
    print("\nSaving Per Class Metric vs. Epoch Plots for {} Clean Datasets vs. Original Dataset ...\n".format(dataset_directory.upper()))
    original_df, cleandf_05, cleandf_10, cleandf_15, cleandf_20 = plot__per_class_metrics__vs__epoch(epochs, dataset_labels, logdir, original_logdir, clean_logdir_05, clean_logdir_10, clean_logdir_15, clean_logdir_20,  'Clean')

    #calls function that calculates distance between clean and triggered per class metrics vectors
    print("\nCalculating Distance between {} Clean vs. Original Per Class Metric Vectors and Saving Results ...\n".format(dataset_directory.upper()))
    clean_perClass_metric_dist_epoch_lst_05 = calculate_distance_for_per_class(count, dataset_labels, original_df, cleandf_05, logdir, '5', 'Clean')
    clean_perClass_metric_dist_epoch_lst_10 = calculate_distance_for_per_class(count, dataset_labels, original_df, cleandf_10, logdir, '10', 'Clean')
    clean_perClass_metric_dist_epoch_lst_15 = calculate_distance_for_per_class(count, dataset_labels, original_df, cleandf_15, logdir, '15', 'Clean')
    clean_perClass_metric_dist_epoch_lst_20 = calculate_distance_for_per_class(count, dataset_labels, original_df, cleandf_20, logdir, '20', 'Clean')

    #calls function that calculates generalization gap for clean datasets
    print("\nCalculating and Saving Generalization Gap for {} Clean Datasets ...\n".format(dataset_directory.upper()))
    cal_generalization_gap(logdir, original_logdir, count, '0', 'Original')
    cal_generalization_gap(logdir, clean_logdir_05, count, '5', 'Clean')
    cal_generalization_gap(logdir, clean_logdir_10, count, '10', 'Clean')
    cal_generalization_gap(logdir, clean_logdir_15, count, '15', 'Clean')
    cal_generalization_gap(logdir, clean_logdir_20, count, '20', 'Clean')

    #calls function to plot history loss and accuracy for triggered vs. original dataset.
    print("\nSaving Loss and Accuracy Plot for {} Triggered Datasets vs. Original Dataset ...\n".format(dataset_directory.upper()))
    plot_histories(history1, triggered_history6, triggered_history7, triggered_history8, triggered_history9, logdir, 'Triggered')

    #calls function to calculate distance for loss and accuracy vectors for triggered vs. original dataset
    print("\nCalculating Distance between Loss and Accuracy Vectors for {} Triggered Datasets vs. Original Dataset ...\n".format(dataset_directory.upper()))
    triggered_metric_dist_epoch_lst_05 = calculate_distance_for_loss_and_acc(logdir, original_logdir, triggered_logdir_05, count, '5', 'Triggered')
    triggered_metric_dist_epoch_lst_10 = calculate_distance_for_loss_and_acc(logdir, original_logdir, triggered_logdir_10, count, '10', 'Triggered')
    triggered_metric_dist_epoch_lst_15 = calculate_distance_for_loss_and_acc(logdir, original_logdir, triggered_logdir_15, count, '15', 'Triggered')
    triggered_metric_dist_epoch_lst_20 = calculate_distance_for_loss_and_acc(logdir, original_logdir, triggered_logdir_20, count, '20', 'Triggered')

    #calls function that generates and saves per class metrics plots for triggered vs. original dataset
    print("\nSaving Per Class Metric vs. Epoch Plots for {} Triggered Datasets vs. Original Dataset ...\n".format(dataset_directory.upper()))
    original_df, triggereddf_05, triggereddf_10, triggereddf_15, triggereddf_20 = plot__per_class_metrics__vs__epoch(epochs, dataset_labels, logdir, original_logdir, triggered_logdir_05, triggered_logdir_10, triggered_logdir_15, triggered_logdir_20,  'Triggered')

    #calls function that calculates distance between clean and triggered per class metrics vectors
    print("\nCalculating Distance between {} Triggered vs. Original Per Class Metric Vectors and Saving Results ...\n".format(dataset_directory.upper()))
    triggered_perClass_metric_dist_epoch_lst_05 = calculate_distance_for_per_class(count, dataset_labels, original_df, triggereddf_05, logdir, '5', 'Triggered')
    triggered_perClass_metric_dist_epoch_lst_10 = calculate_distance_for_per_class(count, dataset_labels, original_df, triggereddf_10, logdir, '10', 'Triggered')
    triggered_perClass_metric_dist_epoch_lst_15 = calculate_distance_for_per_class(count, dataset_labels, original_df, triggereddf_15, logdir, '15', 'Triggered')
    triggered_perClass_metric_dist_epoch_lst_20 = calculate_distance_for_per_class(count, dataset_labels, original_df, triggereddf_20, logdir, '20', 'Triggered')

    #calls function that calculates generalization gap for clean datasets
    print("\nCalculating and Saving Generalization Gap for {} Triggered Datasets ...\n".format(dataset_directory.upper()))
    cal_generalization_gap(logdir, original_logdir, count, '0', 'Original')
    cal_generalization_gap(logdir, triggered_logdir_05, count, '5', 'Triggered')
    cal_generalization_gap(logdir, triggered_logdir_10, count, '10', 'Triggered')
    cal_generalization_gap(logdir, triggered_logdir_15, count, '15', 'Triggered')
    cal_generalization_gap(logdir, triggered_logdir_20, count, '20', 'Triggered')

    #calls function to plot overall dataset metric trajectories as Bar Chart Graph
    print("\nSaving Loss and Accuracy Trajectory Multi Bar Chart Graphs for Original {} vs. Clean {} vs. Triggered {} Datasets ...\n".format(dataset_directory.upper(), dataset_directory.upper(), dataset_directory.upper()))
    generate_multi_bar_graph_training_trajectory_overall_ds(logdir, dataset_directory, '5')
    generate_multi_bar_graph_training_trajectory_overall_ds(logdir, dataset_directory, '10')
    generate_multi_bar_graph_training_trajectory_overall_ds(logdir, dataset_directory, '15')
    generate_multi_bar_graph_training_trajectory_overall_ds(logdir, dataset_directory, '20')

    #calls function to plot perclass metric trajectories as Bar Chart Graph
    print("\nSaving Per-Class Metric Tracjectory Multi Bar Chart Graphs for Original {} vs. Clean {} vs. Triggered {} Datasets ...\n")
    generate_multi_bar_graph_training_trajectory_perclass(logdir, dataset_directory, original_df, cleandf_05, triggereddf_05, '5')
    generate_multi_bar_graph_training_trajectory_perclass(logdir, dataset_directory, original_df, cleandf_10, triggereddf_10, '10')
    generate_multi_bar_graph_training_trajectory_perclass(logdir, dataset_directory, original_df, cleandf_15, triggereddf_15, '15')
    generate_multi_bar_graph_training_trajectory_perclass(logdir, dataset_directory, original_df, cleandf_20, triggereddf_20, '20')

    #calls function that generates clean and triggered bar dist at epoch side by side on same graph
    print("\nSaving Multi Bar Chart Graph that displays Distance calculated between Clean/Triggered {} Metric Vectors vs. Original {} Dataset ...\n".format(dataset_directory.upper(), dataset_directory.upper()))
    get_multi_bar_graph_epochs(logdir, epochs, clean_metric_dist_epoch_lst_05, triggered_metric_dist_epoch_lst_05, '5')
    get_multi_bar_graph_epochs(logdir, epochs, clean_metric_dist_epoch_lst_10, triggered_metric_dist_epoch_lst_10, '10')
    get_multi_bar_graph_epochs(logdir, epochs, clean_metric_dist_epoch_lst_15, triggered_metric_dist_epoch_lst_15, '15')
    get_multi_bar_graph_epochs(logdir, epochs, clean_metric_dist_epoch_lst_20, triggered_metric_dist_epoch_lst_20, '20')

    #calls function that generates clean and triggered bar dist at epoch side by side on same graph
    print("\nSaving Multi Bar Chart Graph that displays Distance calculated between Clean/Triggered {} Per-Class Metric Vectors vs. Original {} Dataset ...\n".format(dataset_directory.upper(), dataset_directory.upper()))
    get_multi_bar_graph_perclass_dist_epochs(logdir, dataset_labels, epochs, clean_perClass_metric_dist_epoch_lst_05, triggered_perClass_metric_dist_epoch_lst_05, '5')
    get_multi_bar_graph_perclass_dist_epochs(logdir, dataset_labels, epochs, clean_perClass_metric_dist_epoch_lst_10, triggered_perClass_metric_dist_epoch_lst_10, '10')
    get_multi_bar_graph_perclass_dist_epochs(logdir, dataset_labels, epochs, clean_perClass_metric_dist_epoch_lst_15, triggered_perClass_metric_dist_epoch_lst_15, '15')
    get_multi_bar_graph_perclass_dist_epochs(logdir, dataset_labels, epochs, clean_perClass_metric_dist_epoch_lst_20, triggered_perClass_metric_dist_epoch_lst_20, '20')

    #Calls function to plot generalization gap as bar chart across epochs to determine how well model is able to generalize
    print("\nSaving Multi Bar Chart Graph that displays Generalization Gap bar for Original, Clean, and Triggered Datasets ...\n")
    generate_multi_bar_graph_gen_gap(logdir, '5')
    generate_multi_bar_graph_gen_gap(logdir, '10')
    generate_multi_bar_graph_gen_gap(logdir, '15')
    generate_multi_bar_graph_gen_gap(logdir, '20')


# **Master CSV of Distance Measure Functions**

In [ ]:
#Function for creating master CSV file that contains all calculated euclidean distances for loss and acc across multiple runs
#Calcuates average, min, and max across all runs.
def master_euclidean_dist_csv(dataset_directory, ds_type):

  main_data_dir = dir.joinpath("logs/scalars/{}/".format(dataset_directory))
  filename = 'Training_Provenance/Calculated_Distance_CSV_Logs/Overall_Dataset/euclidean_distance_between_{}_vs_Original_Loss_and_Acc_Vectors_.csv'.format(ds_type)
  
  print("\nCreating Master CSV Files for Euclidean Distance between {} and Original Metric Vectors Captured for Overall Datasets...\n".format(ds_type))

  subdirectories = []

  if main_data_dir.is_dir():
    subdirectories = [subdir  for subdir in main_data_dir.iterdir() if subdir.is_dir()]

  master_euclidean_dist_df = pd.DataFrame()
  
  for subdir in subdirectories:

    tmp_path = subdir.joinpath(filename)
    tmp_df = pd.read_csv(tmp_path)
    master_euclidean_dist_df = pd.concat([master_euclidean_dist_df, tmp_df])

  master_euclidean_dist_df.sort_values(by=['Run'])
  master_euclidean_dist_csv_filepath = dir.joinpath('logs/scalars/Training_Provenance_Runs/{}/Master_Euclidean_Distance_CSV_Logs/Overall_Dataset/master_euclidean_distance_between_Original_and_{}_metric_vectors.csv'.format(dataset_directory, ds_type))
  master_euclidean_dist_csv_filepath.parent.mkdir(parents=True, exist_ok=True)

  master_euclidean_dist_df.to_csv(master_euclidean_dist_csv_filepath, index = False)

  with open(master_euclidean_dist_csv_filepath, 'a') as outputFile:
      writer = csv.writer(outputFile)
      writer.writerow([''])
      writer.writerow(['Euclidean Distance Average:'])

      for percentage in master_euclidean_dist_df['{} Percentage'.format(ds_type)].unique():

        tmp_df = master_euclidean_dist_df[master_euclidean_dist_df['{} Percentage'.format(ds_type)] == percentage]

        train_loss_dist_avg = tmp_df['Train Loss Euclidean Dist'].mean()
        train_acc_dist_avg = tmp_df['Train Acc Euclidean Dist'].mean()
        val_loss_dist_avg = tmp_df['Val Loss Euclidean Dist'].mean()
        val_acc_dist_avg = tmp_df['Val Acc Euclidean Dist'].mean()

        writer.writerow(['', train_loss_dist_avg, train_acc_dist_avg, val_loss_dist_avg, val_acc_dist_avg, percentage])

      writer.writerows(['',''])
      writer.writerow(['Euclidean Distance Min:'])
      writer.writerow(['Train Loss Euclidean Dist Min', 'Run', 'Train Acc Euclidean Dist Min', 'Run', 'Val Loss Euclidean Dist Min', 'Run',
                       'Val Acc Euclidean Dist Min', 'Run', '{} Percentage'.format(ds_type)])  

      for percentage in master_euclidean_dist_df['{} Percentage'.format(ds_type)].unique():

        tmp_df = master_euclidean_dist_df[master_euclidean_dist_df['{} Percentage'.format(ds_type)] == percentage]

        train_loss_dist_min = tmp_df['Train Loss Euclidean Dist'].min()
        train_acc_dist_min = tmp_df['Train Acc Euclidean Dist'].min()
        val_loss_dist_min = tmp_df['Val Loss Euclidean Dist'].min()
        val_acc_dist_min = tmp_df['Val Acc Euclidean Dist'].min()

        train_loss_dist_min_run = tmp_df[tmp_df['Train Loss Euclidean Dist'] == train_loss_dist_min]['Run'].unique()
        train_acc_dist_min_run = tmp_df[tmp_df['Train Acc Euclidean Dist'] == train_acc_dist_min]['Run'].unique()
        val_loss_dist_min_run = tmp_df[tmp_df['Val Loss Euclidean Dist'] == val_loss_dist_min]['Run'].unique()
        val_acc_dist_min_run = tmp_df[tmp_df['Val Acc Euclidean Dist'] == val_acc_dist_min]['Run'].unique()

        writer.writerow([train_loss_dist_min, train_loss_dist_min_run, train_acc_dist_min, train_acc_dist_min_run,
                         val_loss_dist_min, val_loss_dist_min_run, val_acc_dist_min, val_acc_dist_min_run, percentage])
        
      writer.writerows(['',''])
      writer.writerow(['Euclidean Distance Max:'])
      writer.writerow(['Train Loss Euclidean Dist Max', 'Run', 'Train Acc Euclidean Dist Max', 'Run', 'Val Loss Euclidean Dist Max', 'Run',
                       'Val Acc Euclidean Dist Max', 'Run', '{} Percentage'.format(ds_type)])  


      for percentage in master_euclidean_dist_df['{} Percentage'.format(ds_type)].unique():

        tmp_df = master_euclidean_dist_df[master_euclidean_dist_df['{} Percentage'.format(ds_type)] == percentage]

        train_loss_dist_max = tmp_df['Train Loss Euclidean Dist'].max()
        train_acc_dist_max = tmp_df['Train Acc Euclidean Dist'].max()
        val_loss_dist_max = tmp_df['Val Loss Euclidean Dist'].max()
        val_acc_dist_max = tmp_df['Val Acc Euclidean Dist'].max()

        train_loss_dist_max_run = tmp_df[tmp_df['Train Loss Euclidean Dist'] == train_loss_dist_max]['Run'].unique()
        train_acc_dist_max_run = tmp_df[tmp_df['Train Acc Euclidean Dist'] == train_acc_dist_max]['Run'].unique()
        val_loss_dist_max_run = tmp_df[tmp_df['Val Loss Euclidean Dist'] == val_loss_dist_max]['Run'].unique()
        val_acc_dist_max_run = tmp_df[tmp_df['Val Acc Euclidean Dist'] == val_acc_dist_max]['Run'].unique()

        writer.writerow([train_loss_dist_max, train_loss_dist_max_run, train_acc_dist_max, train_acc_dist_max_run,
                         val_loss_dist_max, val_loss_dist_max_run, val_acc_dist_max, val_acc_dist_max_run, percentage])

  outputFile.close()

  return master_euclidean_dist_df

In [ ]:
#Function for creating master CSV file that contains all calculated euclidean distances for per class metrics across multiple runs
#Calcuates average, min, and max across all runs.
def master_euclidean_perclass_dist_csv(dataset_directory, ds_type):

  main_data_dir = dir.joinpath("logs/scalars/{}/".format(dataset_directory))
  filename = 'Training_Provenance/Calculated_Distance_CSV_Logs/Per_Class/euclidean_dist_per_class_metrics_Original_vs_{}.csv'.format(ds_type)
  
  print("\nCreating Master CSV Files for Euclidean Distance between Clean and Triggered Vectors for Per-Class Labels...\n")

  subdirectories = []

  if main_data_dir.is_dir():
    subdirectories = [subdir  for subdir in main_data_dir.iterdir() if subdir.is_dir()]

  master_perclass_euclidean_dist_df = pd.DataFrame()

  for subdir in subdirectories:

    tmp_path = subdir.joinpath(filename)
    tmp_df = pd.read_csv(tmp_path)
    master_perclass_euclidean_dist_df = pd.concat([master_perclass_euclidean_dist_df, tmp_df])

  master_perclass_euclidean_dist_df.sort_values(by=['Run'])
  master_perclass_dist_csv_filepath = dir.joinpath('logs/scalars/Training_Provenance_Runs/{}/Master_Euclidean_Distance_CSV_Logs/Per_Class/master_euclidean_distance_between_Original_and_{}_Per_Class_Metric_Vectors.csv'.format(dataset_directory, ds_type))
  master_perclass_dist_csv_filepath.parent.mkdir(parents=True, exist_ok=True)

  master_perclass_euclidean_dist_df.to_csv(master_perclass_dist_csv_filepath, index = False)

  with open(master_perclass_dist_csv_filepath,'a') as outputFile:
      writer = csv.writer(outputFile)
      writer.writerow([''])
      writer.writerow(['Euclidean Distance Average:'])
      writer.writerow(['','Label', 'Precision Euclidean Dist Avg', 'Recall Euclidean Dist Avg', 'Support Euclidean Dist Avg', '{} Percentage'.format(ds_type)])

      for label in master_perclass_euclidean_dist_df.Label.unique():

        tmp_df1 = master_perclass_euclidean_dist_df[master_perclass_euclidean_dist_df['Label'] == label]
      
        for percentage in master_perclass_euclidean_dist_df['{} Percentage'.format(ds_type)].unique():

          tmp_df2 = tmp_df1[tmp_df1['{} Percentage'.format(ds_type)] == percentage]
          
          precision_dist_avg = tmp_df2['Precision Euclidean Dist'].mean()
          recall_dist_avg = tmp_df2['Recall Euclidean Dist'].mean()
          support_dist_avg = tmp_df2['Support Euclidean Dist'].mean()
         
          writer.writerow(['', label, precision_dist_avg, recall_dist_avg, support_dist_avg, percentage])

      writer.writerows(['',''])
      writer.writerow(['Euclidean Distance Min:'])
      writer.writerow(['','Label','Precision Euclidean Dist Min', 'Run', 'Recall Euclidean Dist Min', 'Run', 'Support Euclidean Dist Min', 'Run',
                          '{} Percentage'.format(ds_type)])  

      for label in master_perclass_euclidean_dist_df.Label.unique():

        tmp_df1 = master_perclass_euclidean_dist_df[master_perclass_euclidean_dist_df['Label'] == label]

        for percentage in master_perclass_euclidean_dist_df['{} Percentage'.format(ds_type)].unique():

          tmp_df2 = tmp_df1[tmp_df1['{} Percentage'.format(ds_type)] == percentage]
                                           
          precision_dist_min = tmp_df2['Precision Euclidean Dist'].min()
          recall_dist_min = tmp_df2['Recall Euclidean Dist'].min()
          support_dist_min = tmp_df2['Support Euclidean Dist'].min()
        
          precision_dist_min_run = tmp_df2[tmp_df2['Precision Euclidean Dist'] == precision_dist_min]['Run'].unique()
          recall_dist_min_run = tmp_df2[tmp_df2['Recall Euclidean Dist'] == recall_dist_min]['Run'].unique()
          support_dist_min_run = tmp_df2[tmp_df2['Support Euclidean Dist'] == support_dist_min]['Run'].unique()
          
          writer.writerow(['',label, precision_dist_min, precision_dist_min_run, recall_dist_min, recall_dist_min_run,
                         support_dist_min, support_dist_min_run, percentage])
        
      writer.writerows(['',''])
      writer.writerow(['Euclidean Distance Max:'])
      writer.writerow(['','Label', 'Precision Euclidean Dist Max', 'Run', 'Recall Euclidean Dist Max', 'Run', 'Support Euclidean Dist Max', 'Run', '{} Percentage'.format(ds_type)])  

      for label in master_perclass_euclidean_dist_df.Label.unique():

        tmp_df1 = master_perclass_euclidean_dist_df[master_perclass_euclidean_dist_df['Label'] == label]

        for percentage in master_perclass_euclidean_dist_df['{} Percentage'.format(ds_type)].unique():

          tmp_df2 = tmp_df1[tmp_df1['{} Percentage'.format(ds_type)] == percentage]
                                           
          precision_dist_max = tmp_df2['Precision Euclidean Dist'].max()
          recall_dist_max = tmp_df2['Recall Euclidean Dist'].max()
          support_dist_max = tmp_df2['Support Euclidean Dist'].max()
        
          precision_dist_max_run = tmp_df2[tmp_df2['Precision Euclidean Dist'] == precision_dist_max]['Run'].unique()
          recall_dist_max_run = tmp_df2[tmp_df2['Recall Euclidean Dist'] == recall_dist_max]['Run'].unique()
          support_dist_max_run = tmp_df2[tmp_df2['Support Euclidean Dist'] == support_dist_max]['Run'].unique()
          
          writer.writerow(['', label, precision_dist_max, precision_dist_max_run, recall_dist_max, recall_dist_max_run,
                         support_dist_max, support_dist_max_run, percentage])

  outputFile.close()

  return master_perclass_euclidean_dist_df

# **Dataset Specifc Functions**

In [ ]:
#Function for maintaining characteristics and files pertaining to MNIST Dataset.
#Calls experimental functions to run experiments on clean and triggered datasets
def train_CNN_on_MNIST(num_runs, num_epochs, last_count=0):
  
  #MNIST Constants
  dataset_directory = 'mnist'
  mnist_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  mnist_img_height = 28
  mnist_img_width = 28
  mnist_color_mode="grayscale"
  mnist_train_sample_num = 60000
  mnist_batch_size = 32

  Target_Label = "All_to_All"

  #CSV with original dataset
  mnist_clean_train = pd.read_csv(TrojAI_mnist_dir.joinpath("mnist_clean_experiment_train.csv"))
  mnist_clean_test = pd.read_csv(TrojAI_mnist_dir.joinpath("mnist_clean_experiment_test_clean.csv"))

  #CSVs with different percentages of dataset being triggered
  mnist_triggered_05_train = pd.read_csv(TrojAI_mnist_dir.joinpath("mnist_alphatrigger_0.05_experiment_train.csv"))
  mnist_triggered_05_test = pd.read_csv(TrojAI_mnist_dir.joinpath("mnist_alphatrigger_0.05_experiment_test_triggered.csv"))
 
  mnist_triggered_10_train = pd.read_csv(TrojAI_mnist_dir.joinpath("mnist_alphatrigger_0.1_experiment_train.csv"))
  mnist_triggered_10_test = pd.read_csv(TrojAI_mnist_dir.joinpath("mnist_alphatrigger_0.1_experiment_test_triggered.csv"))
 
  mnist_triggered_15_train = pd.read_csv(TrojAI_mnist_dir.joinpath("mnist_alphatrigger_0.15_experiment_train.csv"))
  mnist_triggered_15_test = pd.read_csv(TrojAI_mnist_dir.joinpath("mnist_alphatrigger_0.15_experiment_test_triggered.csv"))
 
  mnist_triggered_20_train = pd.read_csv(TrojAI_mnist_dir.joinpath("mnist_alphatrigger_0.2_experiment_train.csv"))
  mnist_triggered_20_test = pd.read_csv(TrojAI_mnist_dir.joinpath("mnist_alphatrigger_0.2_experiment_test_triggered.csv"))
 
  #calls functions to get directories containing clean data and triggered data
  print("\nGetting Original Clean {} Dataset Directories ...\n".format(dataset_directory.upper()))
  cleanTrain_dir, cleanTest_dir = get_main_clean_dir(TrojAI_mnist_dir, mnist_dir, dataset_directory, mnist_labels, mnist_clean_train, mnist_clean_test)

  print("\nGenerating Augmented Images for Clean {} Dataset Directories ...\n".format(dataset_directory.upper()))  
  #Calls function that find label with the most samples for its class and return num of samples for both Train and Test
  num_trainSamples_per_class, num_testSamples_per_class = get_num_samples_perclass(mnist_clean_train, mnist_clean_test)
  aug_nTrain_samples =  normal_round(max(num_trainSamples_per_class) * .2) + max(num_trainSamples_per_class)
  aug_nTest_samples = normal_round(max(num_testSamples_per_class) * .2) + max(num_testSamples_per_class)

  #Calls function that will generate augmented images and return directorires where augmented images are stored.
  aug_cleanTrain_dir, aug_cleanTest_dir = generate_augmented_datasets(TrojAI_mnist_dir, dataset_directory, cleanTrain_dir, cleanTest_dir, aug_nTrain_samples, aug_nTest_samples, (mnist_img_height, mnist_img_width), mnist_color_mode)

  #Calls function that will retrieve clean experiment directories with augmented images
  print("\nGetting Clean {} Dataset Directories with Augmented Images ...\n".format(dataset_directory.upper()))
  cleanTrain_dir_05, cleanTest_dir_05 = get_clean_dirs_w_aug_images(aug_cleanTrain_dir, aug_cleanTest_dir, dataset_directory, cleanTrain_dir, cleanTest_dir, '5')
  cleanTrain_dir_10, cleanTest_dir_10 = get_clean_dirs_w_aug_images(aug_cleanTrain_dir, aug_cleanTest_dir, dataset_directory, cleanTrain_dir, cleanTest_dir, '10')
  cleanTrain_dir_15, cleanTest_dir_15 = get_clean_dirs_w_aug_images(aug_cleanTrain_dir, aug_cleanTest_dir, dataset_directory, cleanTrain_dir, cleanTest_dir, '15')
  cleanTrain_dir_20, cleanTest_dir_20 = get_clean_dirs_w_aug_images(aug_cleanTrain_dir, aug_cleanTest_dir, dataset_directory, cleanTrain_dir, cleanTest_dir, '20')

  #Calls function that will retrieve triggered experiment directories with triggered images
  print("\nGetting Triggered {} Dataset Directories with Triggered Images ...\n".format(dataset_directory.upper()))
  triggeredTrain_dir_05, triggeredTest_dir_05 = get_triggered_dirs(TrojAI_mnist_dir, cleanTrain_dir, cleanTest_dir, dataset_directory, '5', mnist_triggered_05_train, mnist_triggered_05_test)
  triggeredTrain_dir_10, triggeredTest_dir_10 = get_triggered_dirs(TrojAI_mnist_dir, cleanTrain_dir, cleanTest_dir, dataset_directory, '10', mnist_triggered_10_train, mnist_triggered_10_test)
  triggeredTrain_dir_15, triggeredTest_dir_15 = get_triggered_dirs(TrojAI_mnist_dir, cleanTrain_dir, cleanTest_dir, dataset_directory, '15', mnist_triggered_15_train, mnist_triggered_15_test)
  triggeredTrain_dir_20, triggeredTest_dir_20 = get_triggered_dirs(TrojAI_mnist_dir, cleanTrain_dir, cleanTest_dir, dataset_directory, '20', mnist_triggered_20_train, mnist_triggered_20_test)

  #For loop for running experiments for a set num of runs
  for run in range(num_runs):

    count = last_count + 1 
    last_count = count
    #Creates log directory for given dataset with timestamp
    logdir = dir.joinpath("logs/scalars/{}/".format(dataset_directory) + datetime.now().strftime("%Y%m%d-%H%M%S"))
    logdir.mkdir(parents=True)

    print("\nRun {} for training CNN with {} Datasets".format(count, dataset_directory.upper()))

    #Call function to run clean experiment on original dataset
    #Returns training provenance captured, and clean log directory for original dataset
    history1, clean_logdir = run_clean_experiments(logdir, dataset_directory, mnist_labels, num_epochs, mnist_train_sample_num, cleanTrain_dir, cleanTest_dir, mnist_img_height, mnist_img_width, mnist_color_mode, mnist_batch_size, '0')  
    
    #Call function to run clean experiments on datasets with augmented images. 
    #Num of Augmented Images Added to Original Dataset = Original Dataset x Percentage
    #Returns training provenance captured, and clean log directories for clean datasets with augmented images
    history2, clean_05_logdir = run_clean_experiments(logdir, dataset_directory, mnist_labels, num_epochs, (modified_dataset_train_num_samples(mnist_train_sample_num, 0.05)), cleanTrain_dir_05, cleanTest_dir_05, mnist_img_height, mnist_img_width, mnist_color_mode, mnist_batch_size, '5')
    history3, clean_10_logdir = run_clean_experiments(logdir, dataset_directory, mnist_labels, num_epochs, (modified_dataset_train_num_samples(mnist_train_sample_num, 0.10)), cleanTrain_dir_10, cleanTest_dir_10, mnist_img_height, mnist_img_width, mnist_color_mode, mnist_batch_size, '10')
    history4, clean_15_logdir = run_clean_experiments(logdir, dataset_directory, mnist_labels, num_epochs, (modified_dataset_train_num_samples(mnist_train_sample_num, 0.15)), cleanTrain_dir_15, cleanTest_dir_15, mnist_img_height, mnist_img_width, mnist_color_mode, mnist_batch_size, '15')
    history5, clean_20_logdir = run_clean_experiments(logdir, dataset_directory, mnist_labels, num_epochs, (modified_dataset_train_num_samples(mnist_train_sample_num, 0.20)), cleanTrain_dir_20, cleanTest_dir_20, mnist_img_height, mnist_img_width, mnist_color_mode, mnist_batch_size, '20')

    #Call function to run triggered experiments on datasets with triggered images. 
    #Num of Triggered Images Added to Original Dataset = Original Dataset x Percentage
    #Returns training provenance captured, and triggered log directories for triggered datasets with triggered images
    history6, triggered_05_logdir = run_triggered_experiments(logdir, dataset_directory, mnist_labels, num_epochs, (modified_dataset_train_num_samples(mnist_train_sample_num, 0.05)), '5', triggeredTrain_dir_05, triggeredTest_dir_05, mnist_img_height, mnist_img_width, mnist_color_mode, mnist_batch_size)
    history7, triggered_10_logdir = run_triggered_experiments(logdir, dataset_directory, mnist_labels, num_epochs, (modified_dataset_train_num_samples(mnist_train_sample_num, 0.10)), '10', triggeredTrain_dir_10, triggeredTest_dir_10, mnist_img_height, mnist_img_width, mnist_color_mode, mnist_batch_size)
    history8, triggered_15_logdir = run_triggered_experiments(logdir, dataset_directory, mnist_labels, num_epochs, (modified_dataset_train_num_samples(mnist_train_sample_num, 0.15)), '15', triggeredTrain_dir_15, triggeredTest_dir_15, mnist_img_height, mnist_img_width, mnist_color_mode, mnist_batch_size)
    history9, triggered_20_logdir = run_triggered_experiments(logdir, dataset_directory, mnist_labels, num_epochs, (modified_dataset_train_num_samples(mnist_train_sample_num, 0.20)), '20', triggeredTrain_dir_20, triggeredTest_dir_20, mnist_img_height, mnist_img_width, mnist_color_mode, mnist_batch_size)

    #Call function to generate plots and csvs with calculated distances between original and clean/triggered metrics vectors as well as distance at epoch
    training_prov_detection(dataset_directory, mnist_labels, num_epochs, logdir, clean_logdir, clean_05_logdir, clean_10_logdir, clean_15_logdir, clean_20_logdir, triggered_05_logdir, triggered_10_logdir, triggered_15_logdir, triggered_20_logdir, history1, history2, history3, history4, history5, history6, history7, history8, history9, count)

  #Calls function to generate master euclidean distance csv file for maintaining results over period of runs.
  #Does this for clean experiments and triggered experiments  
  master_euclidean_dist_clean_df = master_euclidean_dist_csv(dataset_directory, 'Clean')
  master_perclass_euclidean_dist_clean_df =  master_euclidean_perclass_dist_csv(dataset_directory, 'Clean')

  master_euclidean_dist_triggered_df = master_euclidean_dist_csv(dataset_directory, 'Triggered')
  master_perclass_euclidean_dist_triggered_df =  master_euclidean_perclass_dist_csv(dataset_directory, 'Triggered')

  #Calls function to generate multi bar chart graph for euclidean distance metrics for overall dataset
  get_multi_bargraph_euclidean_dist_runs_overall_ds(dataset_directory, master_euclidean_dist_clean_df,  master_euclidean_dist_triggered_df, 5)
  get_multi_bargraph_euclidean_dist_runs_overall_ds(dataset_directory, master_euclidean_dist_clean_df,  master_euclidean_dist_triggered_df, 10)
  get_multi_bargraph_euclidean_dist_runs_overall_ds(dataset_directory, master_euclidean_dist_clean_df,  master_euclidean_dist_triggered_df, 15)
  get_multi_bargraph_euclidean_dist_runs_overall_ds(dataset_directory, master_euclidean_dist_clean_df,  master_euclidean_dist_triggered_df, 20)
  
  #Calls function to generate multi bar chart graph for euclidean distance metrics for per class metrics
  get_multi_bargraph_euclidean_dist_runs_perclass(dataset_directory, master_perclass_euclidean_dist_clean_df, master_perclass_euclidean_dist_triggered_df, 5)
  get_multi_bargraph_euclidean_dist_runs_perclass(dataset_directory, master_perclass_euclidean_dist_clean_df, master_perclass_euclidean_dist_triggered_df, 10)
  get_multi_bargraph_euclidean_dist_runs_perclass(dataset_directory, master_perclass_euclidean_dist_clean_df, master_perclass_euclidean_dist_triggered_df, 15)
  get_multi_bargraph_euclidean_dist_runs_perclass(dataset_directory, master_perclass_euclidean_dist_clean_df, master_perclass_euclidean_dist_triggered_df, 20)

  return last_count

In [ ]:
#Function for maintaining characteristics and files pertaining to CIFAR10 Dataset.
#Calls experimental functions to run experiments on clean and triggered datasets
def train_CNN_on_CIFAR10(num_runs, num_epochs, last_count=0):

  #CIFAR10 Constants
  dataset_directory = 'cifar10'
  cifar10_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  cifar10_img_height = 32
  cifar10_img_width = 32
  cifar10_color_mode = "rgb"
  cifar10_train_sample_num = 50000
  cifar10_batch_size = 250

  Target_Label = "4" #Based on targeted label

  #CSV with original dataset
  cifar10_clean_train = pd.read_csv(TrojAI_cifar10_dir.joinpath("cifar10_clean_experiment_train.csv"))
  cifar10_clean_test = pd.read_csv(TrojAI_cifar10_dir.joinpath("cifar10_clean_experiment_test_clean.csv"))

  #CSVs with different percentages of dataset being triggered
  cifar10_triggered_05_train = pd.read_csv(TrojAI_cifar10_dir.joinpath("cifar10_iggothamtrigger_0.05_experiment_train.csv"))
  cifar10_triggered_05_test = pd.read_csv(TrojAI_cifar10_dir.joinpath("cifar10_iggothamtrigger_0.05_experiment_test_triggered.csv"))

  cifar10_triggered_10_train = pd.read_csv(TrojAI_cifar10_dir.joinpath("cifar10_iggothamtrigger_0.1_experiment_train.csv"))
  cifar10_triggered_10_test = pd.read_csv(TrojAI_cifar10_dir.joinpath("cifar10_iggothamtrigger_0.1_experiment_test_triggered.csv"))

  cifar10_triggered_15_train = pd.read_csv(TrojAI_cifar10_dir.joinpath("cifar10_iggothamtrigger_0.15_experiment_train.csv"))
  cifar10_triggered_15_test = pd.read_csv(TrojAI_cifar10_dir.joinpath("cifar10_iggothamtrigger_0.15_experiment_test_triggered.csv"))

  cifar10_triggered_20_train = pd.read_csv(TrojAI_cifar10_dir.joinpath("cifar10_iggothamtrigger_0.2_experiment_train.csv"))
  cifar10_triggered_20_test = pd.read_csv(TrojAI_cifar10_dir.joinpath("cifar10_iggothamtrigger_0.2_experiment_test_triggered.csv"))
  
  #calls functions to get directories containing clean data and triggered data
  print("\nGetting Original Clean {} Dataset Directories ...\n".format(dataset_directory.upper()))
  cleanTrain_dir, cleanTest_dir = get_main_clean_dir(TrojAI_cifar10_dir, cifar10_dir, dataset_directory, cifar10_labels, cifar10_clean_train, cifar10_clean_test)

  print("\nGenerating Augmented Images for Clean {} Dataset Directories ...\n".format(dataset_directory.upper()))  
  #Calls function that find label with the most samples for its class and return num of samples for both Train and Test
  num_trainSamples_per_class, num_testSamples_per_class = get_num_samples_perclass(cifar10_clean_train, cifar10_clean_test)
  aug_nTrain_samples =  normal_round(max(num_trainSamples_per_class) * .2) + max(num_trainSamples_per_class)
  aug_nTest_samples = normal_round(max(num_testSamples_per_class) * .2) + max(num_testSamples_per_class)

  #Calls function that will generate augmented images and return directorires where augmented images are stored.
  aug_cleanTrain_dir, aug_cleanTest_dir = generate_augmented_datasets(TrojAI_cifar10_dir, dataset_directory, cleanTrain_dir, cleanTest_dir, aug_nTrain_samples, aug_nTest_samples, (cifar10_img_height, cifar10_img_width), cifar10_color_mode)

  #Calls function that will retrieve clean experiment directories with augmented images
  print("\nGetting Clean {} Dataset Directories with Augmented Images ...\n".format(dataset_directory.upper()))
  cleanTrain_dir_05, cleanTest_dir_05 = get_clean_dirs_w_aug_images(aug_cleanTrain_dir, aug_cleanTest_dir, dataset_directory, cleanTrain_dir, cleanTest_dir, '5', '4', '5')
  cleanTrain_dir_10, cleanTest_dir_10 = get_clean_dirs_w_aug_images(aug_cleanTrain_dir, aug_cleanTest_dir, dataset_directory, cleanTrain_dir, cleanTest_dir, '10', '4', '5')
  cleanTrain_dir_15, cleanTest_dir_15 = get_clean_dirs_w_aug_images(aug_cleanTrain_dir, aug_cleanTest_dir, dataset_directory, cleanTrain_dir, cleanTest_dir, '15', '4', '5')
  cleanTrain_dir_20, cleanTest_dir_20 = get_clean_dirs_w_aug_images(aug_cleanTrain_dir, aug_cleanTest_dir, dataset_directory, cleanTrain_dir, cleanTest_dir, '20', '4', '5')

  #Calls function that will retrieve triggered experiment directories with triggered images
  print("\nGetting Triggered {} Dataset Directories with Triggered Images ...\n".format(dataset_directory.upper()))
  triggeredTrain_dir_05, triggeredTest_dir_05 = get_triggered_dirs(TrojAI_cifar10_dir, cleanTrain_dir, cleanTest_dir, dataset_directory, '5', cifar10_triggered_05_train, cifar10_triggered_05_test)
  triggeredTrain_dir_10, triggeredTest_dir_10 = get_triggered_dirs(TrojAI_cifar10_dir, cleanTrain_dir, cleanTest_dir, dataset_directory, '10', cifar10_triggered_10_train, cifar10_triggered_10_test)
  triggeredTrain_dir_15, triggeredTest_dir_15 = get_triggered_dirs(TrojAI_cifar10_dir, cleanTrain_dir, cleanTest_dir, dataset_directory, '15', cifar10_triggered_15_train, cifar10_triggered_15_test)
  triggeredTrain_dir_20, triggeredTest_dir_20 = get_triggered_dirs(TrojAI_cifar10_dir, cleanTrain_dir, cleanTest_dir, dataset_directory, '20', cifar10_triggered_20_train, cifar10_triggered_20_test)

  #For loop for running experiments for a set num of runs
  for run in range(num_runs):
  
    count = last_count + 1
    last_count = count

    #Creates log directory for given dataset with timestamp
    logdir = dir.joinpath("logs/scalars/{}/".format(dataset_directory) + datetime.now().strftime("%Y%m%d-%H%M%S"))
    logdir.mkdir(parents=True)

    #Call function to run clean experiment on original dataset
    #Returns main log directory, training provenance captured, and clean log directory for original dataset 
    history1, clean_logdir = run_clean_experiments(logdir, dataset_directory, cifar10_labels, num_epochs, cifar10_train_sample_num, cleanTrain_dir, cleanTest_dir, cifar10_img_height, cifar10_img_width, cifar10_color_mode, cifar10_batch_size, '0')  

    #Call function to run clean experiments on datasets with augmented images. 
    #Num of Augmented Images Added to Original Dataset = Original Dataset x Percentage
    #Returns training provenance captured, and clean log directories for clean datasets with augmented images
    history2, clean_05_logdir = run_clean_experiments(logdir, dataset_directory, cifar10_labels, num_epochs, (modified_dataset_train_num_samples(cifar10_train_sample_num, 0.05)), cleanTrain_dir_05, cleanTest_dir_05, cifar10_img_height, cifar10_img_width, cifar10_color_mode, cifar10_batch_size, '5')
    history3, clean_10_logdir = run_clean_experiments(logdir, dataset_directory, cifar10_labels, num_epochs, (modified_dataset_train_num_samples(cifar10_train_sample_num, 0.10)), cleanTrain_dir_10, cleanTest_dir_10, cifar10_img_height, cifar10_img_width, cifar10_color_mode, cifar10_batch_size, '10')
    history4, clean_15_logdir = run_clean_experiments(logdir, dataset_directory, cifar10_labels, num_epochs, (modified_dataset_train_num_samples(cifar10_train_sample_num, 0.15)), cleanTrain_dir_15, cleanTest_dir_15, cifar10_img_height, cifar10_img_width, cifar10_color_mode, cifar10_batch_size, '15')
    history5, clean_20_logdir = run_clean_experiments(logdir, dataset_directory, cifar10_labels, num_epochs, (modified_dataset_train_num_samples(cifar10_train_sample_num, 0.20)), cleanTrain_dir_20, cleanTest_dir_20, cifar10_img_height, cifar10_img_width, cifar10_color_mode, cifar10_batch_size, '20')

    #Call function to run triggered experiments on datasets with triggered images. 
    #Num of Triggered Images Added to Original Dataset = Original Dataset x Percentage
    #Returns training provenance captured, and triggered log directories for triggered datasets with triggered images
    history6, triggered_05_logdir = run_triggered_experiments(logdir, dataset_directory, cifar10_labels, num_epochs, (modified_dataset_train_num_samples(cifar10_train_sample_num, 0.05)), '5', triggeredTrain_dir_05, triggeredTest_dir_05, cifar10_img_height, cifar10_img_width, cifar10_color_mode, cifar10_batch_size)
    history7, triggered_10_logdir = run_triggered_experiments(logdir, dataset_directory, cifar10_labels, num_epochs, (modified_dataset_train_num_samples(cifar10_train_sample_num, 0.10)), '10', triggeredTrain_dir_10, triggeredTest_dir_10, cifar10_img_height, cifar10_img_width, cifar10_color_mode, cifar10_batch_size)
    history8, triggered_15_logdir = run_triggered_experiments(logdir, dataset_directory, cifar10_labels, num_epochs, (modified_dataset_train_num_samples(cifar10_train_sample_num, 0.15)), '15', triggeredTrain_dir_15, triggeredTest_dir_15, cifar10_img_height, cifar10_img_width, cifar10_color_mode, cifar10_batch_size)
    history9, triggered_20_logdir = run_triggered_experiments(logdir, dataset_directory, cifar10_labels, num_epochs, (modified_dataset_train_num_samples(cifar10_train_sample_num, 0.20)), '20', triggeredTrain_dir_20, triggeredTest_dir_20, cifar10_img_height, cifar10_img_width, cifar10_color_mode, cifar10_batch_size) 

    #Call function to generate plots and csvs with calculated distances between original and clean/triggered metrics vectors as well as distance at epoch
    training_prov_detection(dataset_directory, cifar10_labels, num_epochs, logdir, clean_logdir, clean_05_logdir, clean_10_logdir, clean_15_logdir, clean_20_logdir, triggered_05_logdir, triggered_10_logdir, triggered_15_logdir, triggered_20_logdir, history1, history2, history3, history4, history5, history6, history7, history8, history9, count)
    
  #Calls function to generate master euclidean distance csv file for maintaining results over period of runs.
  #Does this for clean experiments and triggered experiments  
  master_euclidean_dist_clean_df = master_euclidean_dist_csv(dataset_directory, 'Clean')
  master_perclass_euclidean_dist_clean_df =  master_euclidean_perclass_dist_csv(dataset_directory, 'Clean')

  master_euclidean_dist_triggered_df = master_euclidean_dist_csv(dataset_directory, 'Triggered')
  master_perclass_euclidean_dist_triggered_df =  master_euclidean_perclass_dist_csv(dataset_directory, 'Triggered')

  #Calls function to generate multi bar chart graph for euclidean distance metrics for overall dataset
  get_multi_bargraph_euclidean_dist_runs_overall_ds(dataset_directory, master_euclidean_dist_clean_df,  master_euclidean_dist_triggered_df, 5)
  get_multi_bargraph_euclidean_dist_runs_overall_ds(dataset_directory, master_euclidean_dist_clean_df,  master_euclidean_dist_triggered_df, 10)
  get_multi_bargraph_euclidean_dist_runs_overall_ds(dataset_directory, master_euclidean_dist_clean_df,  master_euclidean_dist_triggered_df, 15)
  get_multi_bargraph_euclidean_dist_runs_overall_ds(dataset_directory, master_euclidean_dist_clean_df,  master_euclidean_dist_triggered_df, 20)
  
  #Calls function to generate multi bar chart graph for euclidean distance metrics for per class metrics
  get_multi_bargraph_euclidean_dist_runs_perclass(dataset_directory, master_perclass_euclidean_dist_clean_df, master_perclass_euclidean_dist_triggered_df, 5)
  get_multi_bargraph_euclidean_dist_runs_perclass(dataset_directory, master_perclass_euclidean_dist_clean_df, master_perclass_euclidean_dist_triggered_df, 10)
  get_multi_bargraph_euclidean_dist_runs_perclass(dataset_directory, master_perclass_euclidean_dist_clean_df, master_perclass_euclidean_dist_triggered_df, 15)
  get_multi_bargraph_euclidean_dist_runs_perclass(dataset_directory, master_perclass_euclidean_dist_clean_df, master_perclass_euclidean_dist_triggered_df, 20)

  return last_count

In [ ]:
def modified_dataset_train_num_samples(train_sample_num, added_percentage):
 
 return normal_round(train_sample_num * added_percentage)



# **DEFINE TEST FUNCTION HERE.**

# **TESTING NEW FUNCTIONS HERE.**

# **MAIN FUNCTION DEFINED HERE. CALLS OTHER FUNCTIONS**




In [ ]:
def main():
 
  number_runs = 16
  number_epochs = 10
  last_count = 10

  #print("\nRunning Experiment on MNIST dataset for {} run(s) ...\n".format(number_runs))
  #last_run = train_CNN_on_MNIST(number_runs, number_epochs, last_count) #last run = 30
  #print("Last Run for MNIST: {}".format(last_run))

  print("\nRunning Experiment on CIFAR10 dataset for {} run(s) ...\n".format(number_runs))
  last_run = train_CNN_on_CIFAR10(number_runs, number_epochs, last_count)
  print("Last Run for CIFAR10: {}".format(last_run)) #last run = 
  


# **RUN MAIN FUNCTION HERE.**

In [ ]:
if __name__ == '__main__':
  main()


Running Experiment on CIFAR10 dataset for 16 run(s) ...


Getting Original Clean CIFAR10 Dataset Directories ...


Generating Augmented Images for Clean CIFAR10 Dataset Directories ...



9                        : 100%|███████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 332116.87files/s]


Found 5000 validated image filenames.
Found 5000 validated image filenames.
Found 5000 validated image filenames.
Found 5000 validated image filenames.
Found 5000 validated image filenames.
Found 5000 validated image filenames.
Found 5000 validated image filenames.
Found 5000 validated image filenames.
Found 5000 validated image filenames.
Found 5000 validated image filenames.
Total Augmented images created=  10000
Found 1000 validated image filenames.
Found 1000 validated image filenames.
Found 1000 validated image filenames.
Found 1000 validated image filenames.
Found 1000 validated image filenames.
Found 1000 validated image filenames.
Found 1000 validated image filenames.
Found 1000 validated image filenames.
Found 1000 validated image filenames.
Found 1000 validated image filenames.
Total Augmented images created=  2000

Getting Clean CIFAR10 Dataset Directories with Augmented Images ...


Copying Augmented Images into Clean CIFAR10 Dataset (5% Augmented) Directory ...



9                        : 100%|███████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 638596.83files/s]



Copying Augmented Images into Clean CIFAR10 Dataset (10% Augmented) Directory ...



...


Getting Clean CIFAR10 Training and Test Dataset (20% Augmented) ...

Found 51000 files belonging to 10 classes.
Found 10200 files belonging to 10 classes.

Training CNN Model with Clean CIFAR10 Datasets (20% Augmented) and Logging Metrics ...

Model: "sequential_121"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_726 (Conv2D)         (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_847 (Ba  (None, 32, 32, 32)       128       
 tchNormalization)                                               
                                                                 
 conv2d_727 (Conv2D)         (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_848 (Ba  (None, 32, 32, 32)       128       
 tchNormalization)                                               
     

<ipython-input-29-c6d93e152e13>:32: RuntimeWarning: invalid value encountered in true_divide
  precision[labels[i]] = tp[i] / (tp[i] + fp) # Precision



Getting Triggered CIFAR10 Training and Test Dataset (5% Triggered)...

Found 50250 files belonging to 10 classes.
Found 10249 files belonging to 10 classes.

Training Model with Triggered CIFAR10 Datasets (5% Triggered) and Logging Metrics ...

Model: "sequential_122"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_732 (Conv2D)         (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_854 (Ba  (None, 32, 32, 32)       128       
 tchNormalization)                                               
                                                                 
 conv2d_733 (Conv2D)         (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_855 (Ba  (None, 32, 32, 32)       128       
 tchNormalization)                                               
    

<ipython-input-29-c6d93e152e13>:32: RuntimeWarning: invalid value encountered in true_divide
  precision[labels[i]] = tp[i] / (tp[i] + fp) # Precision



Getting Triggered CIFAR10 Training and Test Dataset (10% Triggered)...

Found 50500 files belonging to 10 classes.
Found 10249 files belonging to 10 classes.

Training Model with Triggered CIFAR10 Datasets (10% Triggered) and Logging Metrics ...

Model: "sequential_123"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_738 (Conv2D)         (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_861 (Ba  (None, 32, 32, 32)       128       
 tchNormalization)                                               
                                                                 
 conv2d_739 (Conv2D)         (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_862 (Ba  (None, 32, 32, 32)       128       
 tchNormalization)                                               
  